# Importación de librerias

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import unicodedata
import random
import numpy as np
from collections import Counter

# Importación de los datos

In [11]:
data = pd.read_excel("C:/Users/felip/Desktop/Trabajo/Clasificación Pain Points/data_abril.xlsx")
data.head()

,Fecha,Respuesta,Diplomado,Version Curso,Modulo,Id,Email,Recuento,Unnamed: 8,Unnamed: 9
0,2021-04-01 00:00:00,.,Manicure,V3,Uñas esculpidas en acrílico y manicure francesa,0035G00001f9zDAQAY,naneryspagan@gmail.com,1,NaN,NaN
1,2021-04-01 00:00:00,.,Mecánica Automotriz,V1,Introducción a la mecánica automotriz,0035G00001guJgDQAU,alemora1999@yahoo.com,1,NaN,NaN
2,2021-04-01 00:00:00,.,Nutrición y Buena Alimentación,V2,Menús ricos y saludables,003f400001Tvdo7AAB,chavezajc@yahoo.com,1,NaN,NaN
3,2021-04-01 00:00:00,.,Nutrición y Salud,V2,Asistencia nutricional,0035G00001f8bJhQAI,marina.hernandez91@gmail.com,1,NaN,NaN
4,2021-04-01 00:00:00,.,Repostería y Pastelería,V2,Pastelería 2,0035G00001gtDmFQAU,martha99vazquez@icloud.com,1,NaN,NaN


In [12]:
### ELEGIR ####
#def concatenar_reposteria(x,y):
#    if "Elétricas" in str(x):
#        return x + " " + y
#    else:
#        return x

In [13]:
### ELEGIR ###
#data["Diplomado"]=data.apply(lambda x: concatenar_reposteria(x["Diplomado"],x["Version Curso"]), axis=1)

In [14]:
### ELEGIR ###
data = data.rename(columns={"Version Curso":"Version"})

In [15]:
lista_diplomado_1=list(data["Diplomado"].unique())

In [16]:
sorted(lista_diplomado_1[:-6])

TypeError: '<' not supported between instances of 'float' and 'str'

# Función que remueve los acentos de las palabras

In [17]:
def limpieza_tildes(x):
    x = unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x

In [18]:
data["Respuesta"]=data["Respuesta"].apply(lambda x: limpieza_tildes(x))

# Loop que remueve caracteres (!,.- etc.)

In [19]:
quitar = "_-,!?;:.\n!\'"
for caracter in quitar:
    data["Respuesta"]=data["Respuesta"].str.replace(caracter, " ")

# Se depura el primer caracter y se pone el texto en minúscula

In [20]:
data["Respuesta"]=data["Respuesta"].str[1:]

In [21]:
data["Respuesta"]=data["Respuesta"].str.lower()

# Parte 1: Función de clasificación de sugerencias 

In [22]:
def clasificador2(x):
    if ("fenomenal" in str(x) or "todo super" in str(x) or "me parece bien" in str(x) or "me parece perfecto" in str(x) or "muy bueno" in str(x) or "muy buena" in str(x) or "muy bien" in str(x) or "esta bien" in str(x) or "felicito" in str(x) or "felicidades" in str(x) or "felicitaciones" in str(x) or "muy util" in str(x) or "muy interesante" in str(x) or "siga igual" in str(x) or "sigan igual" in str(x) or "va bien" in str(x) or "muy bien" in str(x) or "todo va bien" in str(x) or "genial" in str(x) or "encanto" in str(x) or "encantar" in str(x) or "perfecto" in str(x) or "todo bien" in str(x) or "todo muy bien" in str(x) or "exc" in str(x) or "satisfech" in str(x) or "satisfacc" in str(x) or "me gusta " in str(x) or "me gusto" in str(x) or "todo bien" in str(x) or "sigan asi" in str(x) or "maravilloso" in str(x) or "nada que mej" in str(x) or "maravilla" in str(x) or "gracias por" in str(x) or "gracias a" in str(x) or "muchas gracias" in str(x) or "muchas grasias" in str(x) or "feliz" in str(x) or "contento" in str(x)) and not ("no " in str(x) or "video" in str(x) or "imagen" in str(x) or "foto" in str(x)):
        return "NO"
    elif ("ninguno" in str(x) or "ninguna" in str(x) or "n/a" in str(x) or "sin com" in str(x) or "sin sug" in str(x) or "sin opi" in str(x) or "ningun come" in str(x) or "ningun sug" in str(x) or "        " in str(x) or "nada que agregar" in str(x) or "ningun comentario" in str(x) or "nada que decir" in str(x) or "no tengo" in str(x)) and not ("descarga" in str(x) or "docente" in str(x) or "video" in str(x)):
        return "NO"
    elif ("detall" in str(x) or "precis" in str(x) or "especifi" in str(x) or "abarc" in str(x) or "aclara" in str(x) or "puntua" in str(x) or "fondo" in str(x) or "profund" in str(x) or "concre" in str(x) or "especi" in str(x)) and ("lectu" in str(x) or "conteni" in str(x) or "tema" in str(x) or "receta" in str(x) or "formula" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "info" in str(x) or "document" in str(x)):
        return "CO1"
    elif ("exten" in str(x) or "ampli" in str(x) or "grande" in str(x) or "reduci" in str(x) or "corto" in str(x) or "disminu" in str(x) or "largo" in str(x) or "breve" in str(x) or "conciso" in str(x) or "dismin" in str(x) or "satura" in str(x) or "tedio" in str(x)) and ("lectura" in str(x) or "conteni" in str(x) or "tema" in str(x) or "info" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "document" in str(x) or "recet" in str(x)):
        return "CO2"
    elif (" mas " in str(x) or "mayor" in str(x) or "aument" in str(x) or "crec" in str(x) or "increment" in str(x) or "ampli" in str(x) or "anad" in str(x) or "agreg" in str(x) or "poco" in str(x) or "peque" in str(x) or "falt" in str(x) or "limit" in str(x)) and ("material" in str(x) or "libro" in str(x) or "herramienta" in str(x) or "util" in str(x) or "lectura" in str(x)):
        return "CO3"
    elif ("didactico" in str(x) or "pedago" in str(x) or "educativo" in str(x) or "interact" in str(x) or "imagene" in str(x) or "fotos" in str(x) or "dinamico" in str(x)) and not ("subi" in str(x) or "descargar" in str(x) or "envia" in str(x) or "baja" in str(x)):
        return "CO4"
    elif ("descarg" in str(x) or " baja" in str(x) or "consulta" in str(x)) and ("contenid" in str(x) or "video" in str(x) or "audio" in str(x) or "recurso" in str(x) or "pdf" in str(x) or "book" in str(x) or "recet" in str(x) or "plantilla" in str(x) or "material" in str(x)) and ("mas" in str(x) or "tener" in str(x)):
        return "CO5" #mirar
    elif ("sustitu" in str(x) or "opcion" in str(x) or "reempl" in str(x) or "supli" in str(x) or "relevar" in str(x) or "cambi" in str(x)) and ("material" in str(x) or "herramienta" in str(x) or "insumo" in str(x) or "componen" in str(x) or "produc" in str(x)):
        return "CO6"
    elif ("idioma" in str(x) or "ingles" in str(x) or "portugues" in str(x) or "espanol" in str(x) or "castellano" in str(x)):
        return "CO8"
    elif ("repite" in str(x) or "mismo" in str(x) or "repeticion" in str(x) or "aporta" in str(x) or "sirve" in str(x) or "diferente" in str(x) or "igual" in str(x) or "identico" in str(x) or "similar" in str(x)) and ("pdf" in str(x)):
        return "CO9"
    elif ("redaccion" in str(x) or "puntuacion" in str(x) or "puntuacion" in str(x) or "ortogra" in str(x) or "redac" in str(x)) or (("error" in str(x) or "equivo" in str(x)) and "escri" in str(x)):
        return "CO10"
    elif ("informacion" in str(x) or "texto" in str(x) or "pdf" in str(x) or "material" in str(x)) and ("incompleto" in str(x) or "confuso" in str(x) or "incongruente" in str(x) or "contradictorio" in str(x) or "repetid" in str(x) or "claro" in str(x) or "complet" in str(x) or "comprende" in str(x)):
        return "CO11"
    elif ("glosario" in str(x) or "vocabulario" in str(x)) or ("palabra" in str(x) and "clave" in str(x)):
        return "CO12"
    elif ("ejemplos" in str(x) or "ejercicio" in str(x) or "ejercicios" in str(x) or "ejemplo" in str(x) or "practica" in str(x)) and ("real" in str(x) or "aplicar" in str(x) or "aplicado" in str(x) or "vida" in str(x)):
        return "CO13"
    elif ("inicia" in str(x) or "empezar" in str(x) or "principio" in str(x) or "inicio" in str(x) or "primero" in str(x)) and ("lista" in str(x) or "herramientas" in str(x) or "ingredientes" in str(x) or "equipo" in str(x) or "catalogo" in str(x) or "inventario" in str(x)):
        return "CO14"
    elif ("actuali" in str(x) or "renova" in str(x) or "moderniz" in str(x) or "nuev" in str(x) or "reciente" in str(x) or "tendencia" in str(x)) and ("info" in str(x) or "texto" in str(x) or "tema" in str(x) or "leccion" in str(x) or "asunto" in str(x)):
        return "CO15"
    elif ("autoria" in str(x) or "bibliografia" in str(x) or "inventor" in str(x)):
        return "CO16"
    elif ("bibliografia" in str(x) or "texto" in str(x) or "video" in str(x) or "libro" in str(x) or "pagina" in str(x) or "recurso" in str(x) or "web" in str(x)) and ("profundiz" in str(x) or "reflex" in str(x) or "estudi" in str(x) or "indag" in str(x) or "investi" in str(x)):
        return "CO17"
    elif ("resum" in str(x) or "sinte" in str(x) or "sumario" in str(x)) and ("vist" in str(x) or "curso" in str(x) or "modulo" in str(x) or "leccion" in str(x) or "diplomado" in str(x)):
        return "CO18"
    elif ("contenido" in str(x) or "tema" in str(x) or "materia" in str(x)) and ("mas" in str(x) or "aument" in str(x) or "crecim" in str(x) or "anadir" in str(x) or "pusi" in str(x)) and ("ia" in str(x)):
        return "CO19"
    elif ("diseno" in str(x) or "calidad" in str(x) or "bonito" in str(x) or "lindo" in str(x) or "feo" in str(x) or "color" in str(x) or "tipografia" in str(x)) and ("imagen" in str(x) or "foto" in str(x) or "visual" in str(x) or "video" in str(x) or "letra" in str(x) or "fuente" in str(x) or "palabra" in str(x)):
        return "CO21"
    elif ("lenguaje" in str(x) or "idioma" in str(x) or "expresa" in str(x)) and ("dificil" in str(x) or "enredado" in str(x) or "complica" in str(x) or "compren" in str(x) or "entend" in str(x) or "entien" in str(x) or "facil" in str(x) or "senci" in str(x)):
        return "CO22" #mirar
    elif ("mismo" in str(x) or "igual" in str(x)) and ("plataforma" in str(x) or "internet" in str(x) or "fuentes" in str(x) or "pagar" in str(x) or "libro" in str(x) or "youtube" in str(x) or "yutube" in str(x)):
        return "CO23"
    elif ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "conseguir" in str(x) or "buscar" in str(x)) and ("insumos" in str(x) or "material" in str(x) or "herramienta" in str(x) or "utensilio" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "receta" in str(x) or "carne" in str(x) or "azuca" in str(x) or "ingredie" in str(x) or "olla" in str(x) or "cuchillo" in str(x) or "pollo" in str(x)):
        return "CO24"
    elif ("pais" in str(x) or "ciudad" in str(x)):
        return "CO25"    
    elif ("frecuencia" in str(x) or "frecue" in str(x) or "habitua" in str(x) or "corriente" in str(x) or "repetir" in str(x) or "usual" in str(x) or "numero" in str(x) or "cantidad" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV1"
    elif (" hora" in str(x) or "jornada" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "calendario" in str(x) or "tiempo" in str(x) or "flex" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV2"
    elif ("interac" in str(x) or "interrelac" in str(x) or "relacio" in str(x) or "comunicar" in str(x) or "trato" in str(x) or "intercambio" in str(x) or "saludo" in str(x) or "conocer" in str(x)) and ("alumno" in str(x) or "docente" in str(x) or "estudiante" in str(x) or "profesor" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "catedratico" in str(x) or "chef" in str(x)):
        return "CV3"
    elif ("duda" in str(x) or "pregunt" in str(x) or "aclara" in str(x) or "entien" in str(x) or "enten" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV4"
    elif ("presentac" in str(x) or "encuest" in str(x) or "publicida" in str(x) or "propagand" in str(x)) and ("tiempo" in str(x) or "perdida" in str(x) or "valioso" in str(x) or "minuto" in str(x) or " hora" in str(x)):
        return "CV5"
    elif ("diapositiva" in str(x) or "slide" in str(x) or "apuntes" in str(x) or "notas" in str(x) or "anotacion" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV7"
    elif ("formato" in str(x) or "encuesta" in str(x) or "llena" in str(x) or "relle" in str(x) or "completa" in str(x)) and ("concentra" in str(x) or "enfo" in str(x)):
        return "CV8" 
    elif ("informa" in str(x) or "avis" in str(x) or "comunic" in str(x) or "adver" in str(x) or "notific" in str(x) or "anunc" in str(x)) and ("participa" in str(x) or "entra" in str(x) or "acce" in str(x) or "meter" in str(x) or "incorpo" in str(x) or "estar" in str(x)) and ("clase" in str(x) or "modulo" in str(x) or "leccion" in str(x)):
        return "CV9"  
    elif ("confuso" in str(x) or "desordena" in str(x) or "enreda" in str(x) or "dific" in str(x) or "entende" in str(x) or "compli" in str(x) or "complej" in str(x)) and ("horario" in str(x) or " hora" in str(x) or "duracion" in str(x) or "horaria" in str(x) or "calendario" in str(x) or "crono" in str(x)):
        return "CV10"
    elif ("repe" in str(x) or "repiten" in str(x) or "reiter" in str(x) or "reincid" in str(x) or "vuelv" in str(x) or "rehace" in str(x) or " dos " in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV11"
    elif ("graba" in str(x) or "reprod" in str(x) or "guarda" in str(x) or "almac" in str(x) or "archiv" in str(x)) and ("acce" in str(x) or "entr" in str(x) or "ir" in str(x) or "mira" in str(x) or "entra" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV12"
    elif ("inscri" in str(x) or "anotar" in str(x) or "registr" in str(x) or "llenar" in str(x) or "escri" in str(x)) and ("clase" in str(x) or "vivo" in str(x)):
        return "CV13"
    elif ("no se ve" in str(x) or "ver" in str(x) or "mira" in str(x) or "observa" in str(x) or "borroso" in str(x) or "zoom" in str(x) or "entrecortado" in str(x)) and ("clase" in str(x) or "leccion" in str(x) or "lesion" in str(x) or "catedra" in str(x) or "modulo" in str(x)):
        return "CV14"
    elif ("tiempo" in str(x) or "periodo" in str(x) or "duracion" in str(x) or "durar" in str(x) or "lapso" in str(x) or "espacion" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "problema" in str(x) or "inquietud" in str(x) or "cuestion" in str(x)):
        return "CV15"
    elif ("grabada" in str(x) or "filmada" in str(x) or "repeti" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV16"
    elif ("repi" in str(x) or "repet" in str(x) or "reincid" in str(x) or "vuelven" in str(x) or "duplic" in str(x) or "rehac" in str(x) or "mismo" in str(x) or "diferente" in str(x) or "distinto" in str(x)) and ("pdf" in str(x)):
        return "CV17"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x) or "elemento" in str(x) or "pieza" in str(x) or "") and ("explica" in str(x) or "describ" in str(x) or "ensena" in str(x) or "justifi" in str(x)):
        return "CV18"
    elif ("practica" in str(x) or "ejercicio" in str(x) or "prueba" in str(x) or "ensay" in str(x) or "ejecu" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "vivo" in str(x)):
        return "CV19"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("economia" in str(x) or "ahorro" in str(x) or "ingreso" in str(x) or "dinero" in str(x) or "plata" in str(x) or "banco" in str(x) or "moneda" in str(x)):
        return "COV1"    
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "obtenido" in str(x) or "consegui" in str(x) or "encontra" in str(x) or "halla" in str(x) or "busca" in str(x)):
        return "COV2"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("enferm" in str(x) or "dolor" in str(x) or "duele" in str(x) or "muerto" in str(x) or "muerte" in str(x)):
        return "COV3"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("amplia" in str(x) or "prorroga" in str(x) or "aplaza" in str(x) or "suspen" in str(x) or "parar" in str(x) or "retras" in str(x)):
        return "COV4"   
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "DO1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x)):
        return "DO2"
    elif ("comunica" in str(x) or "habla" in str(x) or "conversa" in str(x) or "ver" in str(x) or "what" in str(x) or "chat" in str(x) or "telefo" in str(x) or "celula" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO3"
    elif ("varios" in str(x) or "diversos" in str(x) or "variado" in str(x) or "uno" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO4"
    elif ("llamada" in str(x) or "llama" in str(x) or "presenta" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO5"
    elif ("paciencia" in str(x) or "serenidad" in str(x) or "calma" in str(x) or "tranquilidad" in str(x) or "tranquilo" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "explica" in str(x) or "habla" in str(x)):
        return "DO6"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "replica" in str(x) or "comentario" in str(x) or "alimentacion" in str(x)) and ("actividad" in str(x) or "tarea" in str(x) or "practica" in str(x) or "trabajo" in str(x) or "resultado" in str(x) or "plato" in str(x) or "comida" in str(x)):
        return "DO8"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "comentario" in str(x) or "respond" in str(x) or "respue" in str(x)) and (("equivocad" in str(x) or "errado" in str(x)) or ("nombre" in str(x))):
        return "DO9"
    elif ("calificac" in str(x) or "nota" in str(x) or "puntuacion" in str(x)) and ("baja" in str(x) or "perdi" in str(x) or "perder" in str(x) or "exigente" in str(x) or "riguroso" in str(x)):
        return "DO10"
    elif ("profesional" in str(x) or "cualificado" in str(x) or "cualificado" in str(x) or "competente" in str(x)) and (("personal" in str(x) or "persona" in str(x) or "docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x))): #and ("sepa" in str(x) or "saber" in str(x) or "conocimiento" in str(x) or "tema" in str(x))):
        return "DO11"
    elif ("resolve" in str(x) or "resuel" in str(x) or "aclara" in str(x) or "respond" in str(x)) and ("receta" in str(x) or "practica" in str(x) or "ejercicio" in str(x)):
        return "DO12"
    elif ("retroalimenta" in str(x) or "feedback" in str(x) or "feedbac" in str(x) or "comentario" in str(x) or "resp" in str(x)) and ("video" in str(x)):
        return "DO13"
    elif ("enlace" in str(x) or "externo" in str(x) or "web" in str(x) or "internet" in str(x) or "pagina" in str(x) or "yout" in str(x) or "google" in str(x) or "gogle" in str(x)) and ("busca" in str(x) or "mirar" in str(x) or "investi" in str(x)):
        return "DO14"
    elif ("leer" in str(x) or "lee" in str(x) or "memori" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO15"
    elif ("nervio" in str(x) or "seguridad" in str(x) or "inquieto" in str(x) or "segur" in str(x) or "angustia" in str(x) or "tranqui" in str(x) or "histeric" in str(x) or "calma" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO16"
    elif ("habla" in str(x) or "conversa" in str(x) or "dialoga" in str(x) or "charla" in str(x) or "platica" in str(x) or "decir" in str(x) or "explica" in str(x) or "expresa" in str(x)) and ("lento" in str(x) or "claro" in str(x) or "pausado" in str(x) or "tranquil" in str(x) or "calma" in str(x)):
        return "DO17"
    elif ("viste" in str(x) or "presentacion personal" in str(x) or "elegan" in str(x) or "belleza" in str(x) or "ropa" in str(x) or "traje" in str(x) or "indumentaria" in str(x) or "prenda" in str(x) or "vestuario" in str(x) or "atuendo" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO18"
    elif ("retroaliment" in str(x)):
        return "DO20" #Revisar D20 y DO21 cuales son las diferencias
    elif (("promesa" in str(x) or "prometieron" in str(x) or "ofrecieron" in str(x) or "garantiza" in str(x) or "ofre" in str(x)) and ("vend" in str(x) or "mentira" in str(x) or "venta" in str(x) or "llamaron" in str(x) or "publicidad" in str(x) or "propaganda" in str(x)) or ("fraude" in str(x) or "engano" in str(x) or "estafa" in str(x) or "estafa" in str(x))):
        return "OT1"
    elif ("estoy" in str(x) or "siento" in str(x) or "encuentro" in str(x) or "senti" in str(x) or "tengo" in str(x)) and ("frustr" in str(x) or "trist" in str(x) or "fracas" in str(x) or "amarg" in str(x)):
        return "OT2"
    elif ("nada" in str(x) or "no" in str(x) or "cero" in str(x) or "ninguna" in str(x)) and ("comentario" in str(x) or "opinion" in str(x) or "juicio" in str(x) or "agregar" in str(x) or "decir" in str(x)) and ("posit" in str(x) or "buen" in str(x) or "agregar" in str(x) or "decir" in str(x)):
        return "OT3"
    elif ("motivacion" in str(x) or "motivado" in str(x) or "ganas" in str(x) or "estimulo" in str(x) or "animo" in str(x) or "aliento" in str(x) or "incentivo" in str(x)) and ("estudiar" in str(x) or "hacer l" in str(x) or "realizar l" or "aprender" in str(x) or "practic" in str(x) or "cocina" in str(x) or "medit" in str(x) or "entrar a" in str(x) or "iniciar el" in str(x) or "iniciar la" in str(x) or "empezar el" in str(x) or "empezar la" in str(x)):
        return "OT4"
    elif ("recibi" in str(x) or "envia" in str(x) or "mandado" in str(x) or "mandan" in str(x) or "despacha" in str(x) or "tramit" in str(x) or "dado" in str(x) or "dar" in str(x)) and ("diploma" in str(x) or "credencial" in str(x) or "certifica" in str(x) or "titulo" in str(x)):
        return "OT5"
    elif ("inici" in str(x) or "principio" in str(x) or "comienzo" in str(x) or "comenz" in str(x) or "comiens" in str(x)) and ("apoyo" in str(x) or "ayuda" in str(x) or "respaldo" in str(x) or "colabora" in str(x) or "duda" in str(x) or "pregunta" in str(x) or "inquietud" in str(x)):
        return "OT6"
    elif ("mensaj" in str(x) or "aviso" in str(x) or "whats" in str(x) or "llama" in str(x) or "correo" in str(x) or "celular" in str(x) or "telefono" in str(x)) and ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "plata" in str(x) or "dinero" in str(x)):
        return "OT7"
    elif ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "cobra" in str(x) or "plata" in str(x) or "dinero" in str(x)) and ("inscrito" in str(x) or "tomado" in str(x) or "adscrito" in str(x) or "tomando" in str(x) or "visto" in str(x) or "viendo" in str(x)):
        return "OT8"
    #elif (("soy" in str(x) or "siento" in str(x)) and ("talento" in str(x) or "aptitud" in str(x))) or (("soy malo" in str(x) or "soy bueno" in str(x) or "no tengo" in str(x) or "soy pesimo" in str(x) or "no puedo" in str(x))):
        #return "OT9" #mirar como el no tengo #MIRAR LOS NO TENGO
    elif ("interaccion" in str(x) or "interac" in str(x) or "hablar" in str(x) or "conocer" in str(x) or "charla" in str(x) or "platica" in str(x) or "ver" in str(x) or "videollamada" in str(x) or "reunion" in str(x)) and ("alumno" in str(x) or "estudiante" in str(x) or "companero" in str(x) or "pares" in str(x)):
        return "OT10"
    elif ("reconocimiento" in str(x)) and ("internacional" in str(x)):
        return "OT11"
    elif ("presencial" in str(x) or "fisico" in str(x) or "universidad" in str(x)):
        return "OT12"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x) or "manda" in str(x)) and ("video" in str(x) or "actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "archivo" in str(x)):
        return "PL1"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("navega" in str(x) or "explora" in str(x) or "oprimir" in str(x) or "web" in str(x) or "pagina" in str(x) or "entrar a" in str(x)) and not ("acceso" in str(x)):
        return "PL2"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x)) and ("grande" in str(x) or "pesa" in str(x) or "peso" in str(x)):
        return "PL3"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("acceso" in str(x) or "entrada" in str(x)):
        return "PL4"
    elif ("encuesta" in str(x)) and not ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL5"
    elif ("descarga" in str(x) or "bajar" in str(x) or "baja" in str(x) or "descargo" in str(x) or "download" in str(x) or "guarda" in str(x) or "almacen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "video" in str(x) or "archivo" in str(x)):
        return "PL6"
    elif ("encuesta" in str(x)) and ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL7"
    elif ("la app" in str(x) or "aplicacion" in str(x) or "moodle" in str(x)) or ("pagina" in str(x) and ("funcion" in str(x) or "sencil" in str(x) or "facil" in str(x) or "comod" in str(x))):
        return "PL8"
    elif ("avan" in str(x) or "progres" in str(x) or "aprobad" in str(x) or "pase" in str(x) or "evolucion" in str(x) or "cumplid" in str(x) or "realiz" in str(x) or "realis" in str(x) or "nota" in str(x) or "califica" in str(x)) and ("muestra" in str(x) or "regist" in str(x) or "exhibe" in str(x) or "present" in str(x) or "revela" in str(x) or "indica" in str(x) or "ver" in str(x) or "veo" in str(x) or "observa" in str(x)):
        return "PL9"
    elif ("funciona" in str(x) or "sirve" in str(x) or "desemp" in str(x) or "march" in str(x) or "traba" in str(x) or "trava" in str(x) or "estorba" in str(x) or "error" in str(x) or "falla" in str(x) or "fallo" in str(x) or "congela" in str(x) or "pasma" in str(x)) and ("material" in str(x) or "interac" in str(x) or "video" in str(x) or "tarjeta" in str(x) or "animado" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "animacion" in str(x) or "imagen" in str(x) or "grabacion" in str(x)):
        return "PL10"
    elif ("link" in str(x) or "enlace" in str(x) or "boton" in str(x) or "oprim" in str(x) or "hipervinculo" in str(x)) and ("descar" in str(x) or "bajar" in str(x) or "download" in str(x) or "materia" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "video" in str(x) or "receta" in str(x)) and not ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL11"
    elif ("clase" in str(x) or "sesion" in str(x)) and ("traba" in str(x) or "carga" in str(x) or "borra" in str(x) or "funciona" in str(x) or "escucha" in str(x) or "oye" in str(x) or "lenta" in str(x) or "lento" in str(x) or "congel" in str(x)):
        return "PL12"
    elif ("chat" in str(x)) or (("mensaje" in str(x)) and ("app" in str(x) or "aplicacion" in str(x))):
        return "PL13"
    elif ("notifica" in str(x) or "alerta" in str(x) or "señal" in str(x) or "aviso" in str(x)) and ("plataforma" in str(x) or "moodle" in str(x) or "app" in str(x) or "aplicacion" in str(x) or "pagina" in str(x) or "internet" in str(x)):
        return "PL14"
    elif ("reprod" in str(x) or "abri" in str(x) or "desple" in str(x) or "abre" in str(x) or "inic" in str(x) or "escuch" in str(x) or "oir" in str(x) or "ver" in str(x)) and ("video" in str(x) or "audio" in str(x) or "multimedia" in str(x) or "sonido" in str(x)) and ("dispositivo" in str(x) or "celular" in str(x) or "telefo" in str(x) or "computa" in str(x) or "pc" in str(x) or "equipo" in str(x) or "iphone" in str(x) or "android" in str(x) or "mac" in str(x) or "tablet" in str(x) or "pad" in str(x)):
        return "PL15"
    elif ("congela" in str(x) or " traba " in str(x) or "se pone lenta" in str(x) or "esta lenta" in str(x) or "esta muy lenta" in str(x) or "pantalla" in str(x) or "trava" in str(x) or "esta lento" in str(x) or "va lenta" in str(x)):
        return "PL16"
    elif ("enlace" in str(x) or "link" in str(x) or "hipervinculo" in str(x) or "boton" in str(x) or "oprim" in str(x)) and ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL17"  
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "SE1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x) or "atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("linea" in str(x) or "numero" in str(x) or "what" in str(x) or "facebook" in str(x) or "instagram" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)):
        return "SE2"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)) and ("otra area" in str(x) or "otro diplomado" in str(x) or "no sabe" in str(x) or "no tiene conocimiento" in str(x)):
        return "SE3"
    elif ("indu" in str(x)) and ("personal" in str(x) or "individual" in str(x) or "propio" in str(x) or "particular" in str(x) or "basico" in str(x) or "intermedio" in str(x) or "avanzado" in str(x) or "pc" in str(x) or "internet" in str(x) or "utilizar" in str(x) or "web" in str(x)):
        return "SE4"
    elif ("indu" in str(x)) and ("recibi" in str(x) or "dieron" in str(x) or "daron" in str(x) or "no pude" in str(x)):
        return "SE5"
    elif ("indu" in str(x)) and ("explica" in str(x) or "ensen" in str(x) or "aclara" in str(x) or "manifest" in str(x)):
        return "SE6"
    elif ("pacien" in str(x) or "seren" in str(x) or "tranquil" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("explica" in str(x) or "ensena" in str(x)):
        return "SE7"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "tutor" in str(x)) and ("contacto" in str(x) or "numero" in str(x) or "telefono" in str(x) or "info de contacto" in str(x) or "informacion de contacto" in str(x) or "informacion para contactarse" in str(x)):
        return "SE8"
    elif ("llamada" in str(x) or "mensaje" in str(x) or "llama" in str(x)) and ("mucha" in str(x) or "demasia" in str(x) or "cantidad" in str(x) or "exceso" in str(x) or "monton" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "dia" in str(x)):
        return "SE9"
    elif ("termine" in str(x) or "acab" in str(x) or "complet" in str(x) or "conclu" in str(x) or "finali" in str(x)) and ("modulo" in str(x) or "curso" in str(x) or "diplomado" in str(x) or "escuela" in str(x) or "especiali" in str(x) or "carrera" in str(x)):
        return "SE10"
    elif ("seguimiento" in str(x) or "atencion" in str(x) or "atend" in str(x) or "preocup" in str(x)) and ("mas" in str(x) or "mayor" in str(x) or "aument" in str(x)):
        return "SE11"  
    elif ("duda" in str(x) or "pregunta" in str(x) or "cuestion" in str(x)) and ("esfuer" in str(x) or "interes" in str(x) or "ganas" in str(x)) and not ("interesante" in str(x)):
        return "SE12"
    elif ("profundizar" in str(x)) and ("temas" in str(x) or "asunto" in str(x) or "cosa" in str(x) or "agenda" in str(x) or "inscribir" in str(x) or "usar" in str(x) or "uso" in str(x) or "agendar clase" in str(x) or "plataforma" in str(x) or "retroalimentacion" in str(x)):
        return "SE13"
    elif ("tiempo" in str(x) or "tarda" in str(x) or "demora" in str(x)) and ("llamada" in str(x) or "llama" in str(x) or "chat" in str(x) or "spar" in str(x) or "contesta" in str(x) or "resp" in str(x) or "ayuda" in str(x) or "apoyo" in str(x) or "acompa" in str(x)):
        return "SE14"     
    elif ("nunca" in str(x) or "jamas" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "respuesta" in str(x) or "colaboracion" in str(x) or "retroalimentacion" in str(x) or "contest" in str(x) or "llama" in str(x)):
        return "SE15"
    elif ("tiempo" in str(x) or "duracion" in str(x) or "dura" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)) and ("termina" in str(x) or "acaba" in str(x) or "culmin" in str(x) or "completa" in str(x) or "finali" or str(x) or "conclu" in str(x)):
        return "TD1"
    elif ("tiempo" in str(x) or "duracion" in str(x) or " hora" in str(x) or " dia" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("entrega" in str(x) or "subi" in str(x) or "envia" in str(x) or " manda" in str(x)):
        return "TD2"
    elif ("pausa" in str(x) or "congela" in str(x) or "parar" in str(x) or "vaciones" in str(x) or "enfermedad" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)):
        return "TD3"
    elif ("menos" in str(x) or "no mas" in str(x) or "muchas" in str(x) or "disminu" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR1"
    elif ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR2"
    elif ("sencill" in str(x) or "facil" in str(x) or "simple" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR3"
    elif ("tecnic" in str(x) or "teoric" in str(x) or "aplica" in str(x) or "videa real" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR4"
    elif ("dificil" in str(x) or "complica" in str(x) or "complej" in str(x) or "imposib" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR5"
    elif ("extens" in str(x) or "largo" in str(x) or "grande" in str(x) or "corto" in str(x) or "amplio" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR6"
    elif ("foto" in str(x) or "fotografia" in str(x) or "imagen" in str(x) or "prueba" in str(x)) and ("much" in str(x) or "demasi" in str(x) or "excesiv" in str(x) or "poco" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR7"
    elif ("pregunta" in str(x) or "piden" in str(x) or "pedir" in str(x) or "exig" in str(x) or "solici" in str(x)) and ("se vieron" in str(x) or "se abordaron" in str(x) or "no explic" in str(x) or "no ensen" in str(x)):
        return "PR8"
    elif ("instrucc" in str(x) or "paso" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("clar" in str(x) or "confus" in str(x)):
        return "PR9"
    elif ("indice" in str(x) or "clasificacion" in str(x) or "seguimiento" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR11"
    elif ("tiempo" in str(x) or "espacio" in str(x) or "periodo" in str(x) or "dias" in str(x) or " hora" in str(x)) and ("real" in str(x) or "coincid" in str(x) or "igual" in str(x) or "en verdad" in str(x)):
        return "PR12"
    elif ("formato" in str(x) or "amigable" in str(x) or "forma" in str(x) or "disen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR13"
    elif ("poder" in str(x) or "deberia" in str(x) or "seria" in str(x) or "opcion" in str(x) or "alternativ" in str(x) or "eleg" in str(x))  and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "audio" in str(x) or "grabac" in str(x)):
        return "PR14"
    elif ("examen" in str(x) or "ejercicio" in str(x) or "test" in str(x) or "evalua" in str(x)) or ("confus" in str(x) or "confun" in str(x)):
        return "EC1"
    elif ("respuesta bien" in str(x) or "respuesta esta mal" in str(x) or "respuesta no coincide" in str(x) or "respuesta mal" in str(x) or "respuesta bien" in str(x) or "marque la respuesta" in str(x) or "marco la respuesta" in str(x) or "marco la opcion" in str(x)):
        return "EC1"
    elif ("instruccio" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("entend" in str(x) or "entiend" in str(x) or "comprend" in str(x)):
        return "EC2"
    elif ("pregun" in str(x)) and ("no explic" in str(x) or "no die" or "no hi" or "no est"):
        return "EC3"
    elif ("preguntas son" in str(x) or "preguntas debe" in str(x) or "preguntas mas" in str(x) or "preguntas menos" in str(x)) and ("dificil" in str(x) or "facil" in str(x) or "comple" in str(x) or "teor" in str(x) or "tecnic" in str(x)):
        return "EC4"
    elif ("mas eva" in str(x) or "mas eje" in str(x) or "mas cues" in str(x)):
        return "EC5"
    elif ("retroal" in str(x)) and ("evalua" in str(x) or "cuestion" in str(x)):
        return "EC6"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "amplia" in str(x)):
        return "RE1" #mas miremos a ver si se quita
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("ingrediente" in str(x) or "alimento" in str(x) or "comida" in str(x) or "componente" in str(x)):
        return "RE2" #mirar porque está repetida ahí arribe(
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("utensilio" in str(x) or "util" in str(x) or "uten" in str(x) or "herramienta" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "oya" in str(x) or "sarten" in str(x) or "licuador" in str(x) or "cocina" in str(x) or "elemento" in str(x) or "tabla" in str(x) or "rallador" in str(x) or "pelador" in str(x) or "molde" in str(x) or "cacerola" in str(x)):
        return "RE3"
    elif (" tips" in str(x) or "truco" in str(x) or "atajo" in str(x) or "consejo" in str(x) or "recomenda" in str(x) or "sugerenci" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE5"
    elif ("actual" in str(x) or " nuev" in str(x) or " tenden" in str(x) or "presente" in str(x) or "vigente" in str(x) or "moda" in str(x) or "ultimas" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE6"
    elif ("porq" in str(x) or "por q" in str(x) or "motivo" in str(x) or "razon" in str(x) or "motivacion" in str(x) or "fundamento" in str(x) or "finalidad" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE7"
    elif ("receta" in str(x) or "formula" in str(x)) and ("especif" in str(x) or "distint" in str(x) or "especial" in str(x) or "particu" in str(x) or "diferen" in str(x) or "paso a paso" in str(x)):
        return "RE11"
    elif ("gramo" in str(x) or " tasa " in str(x) or " taza " in str(x) or "onza" in str(x) or " onsa" in str(x) or "kilo" in str(x) or "libra" in str(x)):
        return "RE9"
    elif ("receta" in str(x) or "formula" in str(x)) and ("descar" in str(x) or " baja" in str(x)):
        return "RE10"
    elif ("receta" in str(x) or "formula" in str(x)) and ("igual" in str(x) or "coinci" in str(x) or "concord" in str(x) or "similar" in str(x) or "equival" in str(x) or "mismo" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "clase" in str(x)):
        return "RE12"
    elif ("paso a paso" in str(x) or "congruen" in str(x) or "cohere" in str(x)) and ("recet" in str(x) or "formula" in str(x)):
        return "RE13"
    elif ("tamano" in str(x) or "medida" in str(x) or "dimensio" in str(x) or "talla" in str(x) or "grande" in str(x) or "peque" in str(x)) and ("molde" in str(x) or "olla" in str(x) or "cacerola" in str(x) or "cuchillo" in str(x) or "tabla" in str(x) or "rallado" in str(x)):
        return "RE14"
    elif ("receta" in str(x)) and ("rubric" in str(x)):
        return "RE15"
    elif ("receta" in str(x) or "formula" in str(x)) and ("teoria" in str(x)):
        return "RE16"
    elif ("receta" in str(x) or "formula" in str(x) or "plato" in str(x) or "comida" in str(x) or "postre" in str(x)) and ("tiempo" in str(x) or "minuto" in str(x) or "hora" in str(x) or "duracion" in str(x) or "segundo" in str(x)):
        return "RE17"
    elif ("receta" in str(x) or "plato" in str(x) or "plato" in str(x) or "postre" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "pastel" in str(x) or "parrillada" in str(x) or "asado" in str(x)) and ("resultado" in str(x) or "final" in str(x) or "producto" in str(x) or "feo" in str(x) or "sabor" in str(x) or "quedo mal" in str(x) or "se ve mal" in str(x) or " sabe" in str(x) or "se ve feo" in str(x) or "no sale" in str(x) or "no sirve" in str(x)):
        return "RE19"
    elif ("receta" in str(x) or "formula" in str(x)) and ("diseno" in str(x) or "fotografia" in str(x) or " foto" in str(x) or " imagen" in str(x)):
        return "RE20"
    elif ("utensilio" in str(x) or "instrumento" in str(x) or "herramienta" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "molde" in str(x) or "rallador" in str(x) or "tijera" in str(x) or "olla" in str(x) or "batidor" in str(x) or "vaso" in str(x)) and ("explica" in str(x) or "ilustr" in str(x) or "ensen" in str(x) or "indica" in str(x) or "da clases" in str(x) or "educar" in str(x)):
        return "RE21"
    elif ("receta" in str(x) or "formula" in str(x) or "alimento" in str(x) or "comida" in str(x) or "bandeja" in str(x) or "parrill" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "arroz" in str(x) or "postre" in str(x) or "torta" in str(x)) and ("difici" in str(x) or "complej" in str(x) or "complica" in str(x) or "imposibl" in str(x) or "arduo" in str(x) or "fácil" in str(x) or "sencill" in str(x) or "simple" in str(x)):
        return "RE22"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovisu" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and not ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)) :
        return "VI1"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)):
        return "VI2"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("pasos" in str(x) or "proceso" in str(x)):
        return "VI3"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("texto" in str(x) or "escrito" in str(x)):
        return "VI4"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("interact" in str(x)):
        return "VI5"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("ia" in str(x) or "tema" in str(x) or "topic" in str(x) or "materia" in str(x)):
        return "VI6"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mismo" in str(x) or "misma" in str(x) or "ident" in str(x) or "igual" in str(x) or "equivalente" in str(x)):
        return "VI7"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x) or "imagen" in str(x) or "foto" in str(x)) and (" mas " in str(x)):
        return "VI8"
    elif ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)):
        return "VI9"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("calidad" in str(x) or "borroso" in str(x) or "no se ve bien" in str(x) or "imagen" in str(x) or "audio" in str(x) or "volumen" in str(x) or "sonido" in str(x) or "oscuro" in str(x) or "gris" in str(x)):
        return "VI11"
    elif ("set" in str(x) or " estudio" in str(x) or "cocina" in str(x) or "lugar" in str(x) or "sitio" in str(x)) and ("limpio" in str(x) or "aseado" in str(x) or "pulcro" in str(x) or "higienico" in str(x) or "sucio" in str(x)):
        return "VI12"
    elif ("video" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "VI13"
    elif ("video" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x)):
        return "VI14"
    elif ("video" in str(x)) and ("experiencia" in str(x) or "truco" in str(x) or "consejo" in str(x) or "sugerencia" in str(x) or "real" in str(x) or "solucion" in str(x) or "reparar" in str(x)):
        return "VI15"
    elif ("subtitulo" in str(x)):
        return "VI16"
    elif ("video" in str(x)) and ("actual" in str(x) or "reciente" in str(x)):
        return "VI17"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "AU1"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x) or "dura" in str(x)):
        return "AU2"
    elif ("audio" in str(x) or "sonido" in str(x) or "volumen" in str(x)) and ("calidad" in str(x) or "volumen" in str(x) or "escuch" in str(x) or "bajo" in str(x) or "alto" in str(x) or "debil" in str(x) or "ruido" in str(x)):
        return "AU3"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "poco" in str(x) or "increment" in str(x)):
        return "AU5"
    elif("audio" in str(x) or "sonido" in str(x) or "ruido" in str(x)):
        return "AU4"
    else:
        return "-"

In [23]:
def clasificador2_1(x):
    if ("fenomenal" in str(x) or "todo super" in str(x) or "me parece bien" in str(x) or "me parece perfecto" in str(x) or "muy bueno" in str(x) or "muy buena" in str(x) or "muy bien" in str(x) or "esta bien" in str(x) or "felicito" in str(x) or "felicidades" in str(x) or "felicitaciones" in str(x) or "muy util" in str(x) or "muy interesante" in str(x) or "siga igual" in str(x) or "sigan igual" in str(x) or "va bien" in str(x) or "muy bien" in str(x) or "todo va bien" in str(x) or "genial" in str(x) or "encanto" in str(x) or "encantar" in str(x) or "perfecto" in str(x) or "todo bien" in str(x) or "todo muy bien" in str(x) or "exc" in str(x) or "satisfech" in str(x) or "satisfacc" in str(x) or "me gusta " in str(x) or "me gusto" in str(x) or "todo bien" in str(x) or "sigan asi" in str(x) or "maravilloso" in str(x) or "nada que mej" in str(x) or "maravilla" in str(x) or "gracias por" in str(x) or "gracias a" in str(x) or "muchas gracias" in str(x) or "muchas grasias" in str(x) or "feliz" in str(x) or "contento" in str(x)) and not ("no " in str(x) or "video" in str(x) or "imagen" in str(x) or "foto" in str(x)):
        return "NO"
    elif ("ninguno" in str(x) or "ninguna" in str(x) or "n/a" in str(x) or "sin com" in str(x) or "sin sug" in str(x) or "sin opi" in str(x) or "ningun come" in str(x) or "ningun sug" in str(x) or "        " in str(x) or "nada que agregar" in str(x) or "ningun comentario" in str(x) or "nada que decir" in str(x) or "no tengo" in str(x)) and not ("descarga" in str(x) or "docente" in str(x) or "video" in str(x)):
        return "NO"
    elif ("detall" in str(x) or "precis" in str(x) or "especifi" in str(x) or "abarc" in str(x) or "aclara" in str(x) or "puntua" in str(x) or "fondo" in str(x) or "profund" in str(x) or "concre" in str(x) or "especi" in str(x)) and ("lectu" in str(x) or "conteni" in str(x) or "tema" in str(x) or "receta" in str(x) or "formula" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "info" in str(x) or "document" in str(x)):
        return "CO1"
    elif ("exten" in str(x) or "ampli" in str(x) or "grande" in str(x) or "reduci" in str(x) or "corto" in str(x) or "disminu" in str(x) or "largo" in str(x) or "breve" in str(x) or "conciso" in str(x) or "dismin" in str(x) or "satura" in str(x) or "tedio" in str(x)) and ("lectura" in str(x) or "conteni" in str(x) or "tema" in str(x) or "info" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "document" in str(x) or "recet" in str(x)):
        return "CO2"
    elif (" mas " in str(x) or "mayor" in str(x) or "aument" in str(x) or "crec" in str(x) or "increment" in str(x) or "ampli" in str(x) or "anad" in str(x) or "agreg" in str(x) or "poco" in str(x) or "peque" in str(x) or "falt" in str(x) or "limit" in str(x)) and ("material" in str(x) or "libro" in str(x) or "herramienta" in str(x) or "util" in str(x) or "lectura" in str(x)):
        return "CO3"
    elif ("didactico" in str(x) or "pedago" in str(x) or "educativo" in str(x) or "interact" in str(x) or "imagene" in str(x) or "fotos" in str(x) or "dinamico" in str(x)) and not ("subi" in str(x) or "descargar" in str(x) or "envia" in str(x) or "baja" in str(x)):
        return "CO4"
    elif ("descarg" in str(x) or " baja" in str(x) or "consulta" in str(x)) and ("contenid" in str(x) or "video" in str(x) or "audio" in str(x) or "recurso" in str(x) or "pdf" in str(x) or "book" in str(x) or "recet" in str(x) or "plantilla" in str(x) or "material" in str(x)) and ("mas" in str(x) or "tener" in str(x)):
        return "CO5" #mirar
    elif ("sustitu" in str(x) or "opcion" in str(x) or "reempl" in str(x) or "supli" in str(x) or "relevar" in str(x) or "cambi" in str(x)) and ("material" in str(x) or "herramienta" in str(x) or "insumo" in str(x) or "componen" in str(x) or "produc" in str(x)):
        return "CO6"
    elif ("idioma" in str(x) or "ingles" in str(x) or "portugues" in str(x) or "espanol" in str(x) or "castellano" in str(x)):
        return "CO8"
    elif ("repite" in str(x) or "mismo" in str(x) or "repeticion" in str(x) or "aporta" in str(x) or "sirve" in str(x) or "diferente" in str(x) or "igual" in str(x) or "identico" in str(x) or "similar" in str(x)) and ("pdf" in str(x)):
        return "CO9"
    elif ("redaccion" in str(x) or "puntuacion" in str(x) or "puntuacion" in str(x) or "ortogra" in str(x) or "redac" in str(x)) or (("error" in str(x) or "equivo" in str(x)) and "escri" in str(x)):
        return "CO10"
    elif ("informacion" in str(x) or "texto" in str(x) or "pdf" in str(x) or "material" in str(x)) and ("incompleto" in str(x) or "confuso" in str(x) or "incongruente" in str(x) or "contradictorio" in str(x) or "repetid" in str(x) or "claro" in str(x) or "complet" in str(x) or "comprende" in str(x)):
        return "CO11"
    elif ("glosario" in str(x) or "vocabulario" in str(x)) or ("palabra" in str(x) and "clave" in str(x)):
        return "CO12"
    elif ("ejemplos" in str(x) or "ejercicio" in str(x) or "ejercicios" in str(x) or "ejemplo" in str(x) or "practica" in str(x)) and ("real" in str(x) or "aplicar" in str(x) or "aplicado" in str(x) or "vida" in str(x)):
        return "CO13"
    elif ("inicia" in str(x) or "empezar" in str(x) or "principio" in str(x) or "inicio" in str(x) or "primero" in str(x)) and ("lista" in str(x) or "herramientas" in str(x) or "ingredientes" in str(x) or "equipo" in str(x) or "catalogo" in str(x) or "inventario" in str(x)):
        return "CO14"
    elif ("actuali" in str(x) or "renova" in str(x) or "moderniz" in str(x) or "nuev" in str(x) or "reciente" in str(x) or "tendencia" in str(x)) and ("info" in str(x) or "texto" in str(x) or "tema" in str(x) or "leccion" in str(x) or "asunto" in str(x)):
        return "CO15"
    elif ("autoria" in str(x) or "bibliografia" in str(x) or "inventor" in str(x)):
        return "CO16"
    elif ("bibliografia" in str(x) or "texto" in str(x) or "video" in str(x) or "libro" in str(x) or "pagina" in str(x) or "recurso" in str(x) or "web" in str(x)) and ("profundiz" in str(x) or "reflex" in str(x) or "estudi" in str(x) or "indag" in str(x) or "investi" in str(x)):
        return "CO17"
    elif ("resum" in str(x) or "sinte" in str(x) or "sumario" in str(x)) and ("vist" in str(x) or "curso" in str(x) or "modulo" in str(x) or "leccion" in str(x) or "diplomado" in str(x)):
        return "CO18"
    elif ("contenido" in str(x) or "tema" in str(x) or "materia" in str(x)) and ("mas" in str(x) or "aument" in str(x) or "crecim" in str(x) or "anadir" in str(x) or "pusi" in str(x)) and ("ia" in str(x)):
        return "CO19"
    elif ("diseno" in str(x) or "calidad" in str(x) or "bonito" in str(x) or "lindo" in str(x) or "feo" in str(x) or "color" in str(x) or "tipografia" in str(x)) and ("imagen" in str(x) or "foto" in str(x) or "visual" in str(x) or "video" in str(x) or "letra" in str(x) or "fuente" in str(x) or "palabra" in str(x)):
        return "CO21"
    elif ("lenguaje" in str(x) or "idioma" in str(x) or "expresa" in str(x)) and ("dificil" in str(x) or "enredado" in str(x) or "complica" in str(x) or "compren" in str(x) or "entend" in str(x) or "entien" in str(x) or "facil" in str(x) or "senci" in str(x)):
        return "CO22" #mirar
    elif ("mismo" in str(x) or "igual" in str(x)) and ("plataforma" in str(x) or "internet" in str(x) or "fuentes" in str(x) or "pagar" in str(x) or "libro" in str(x) or "youtube" in str(x) or "yutube" in str(x)):
        return "CO23"
    elif ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "conseguir" in str(x) or "buscar" in str(x)) and ("insumos" in str(x) or "material" in str(x) or "herramienta" in str(x) or "utensilio" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "receta" in str(x) or "carne" in str(x) or "azuca" in str(x) or "ingredie" in str(x) or "olla" in str(x) or "cuchillo" in str(x) or "pollo" in str(x)):
        return "CO24"
    elif ("pais" in str(x) or "ciudad" in str(x)):
        return "CO25"    
    elif ("frecuencia" in str(x) or "frecue" in str(x) or "habitua" in str(x) or "corriente" in str(x) or "repetir" in str(x) or "usual" in str(x) or "numero" in str(x) or "cantidad" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV1"
    elif (" hora" in str(x) or "jornada" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "calendario" in str(x) or "tiempo" in str(x) or "flex" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV2"
    elif ("interac" in str(x) or "interrelac" in str(x) or "relacio" in str(x) or "comunicar" in str(x) or "trato" in str(x) or "intercambio" in str(x) or "saludo" in str(x) or "conocer" in str(x)) and ("alumno" in str(x) or "docente" in str(x) or "estudiante" in str(x) or "profesor" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "catedratico" in str(x) or "chef" in str(x)):
        return "CV3"
    elif ("duda" in str(x) or "pregunt" in str(x) or "aclara" in str(x) or "entien" in str(x) or "enten" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV4"
    elif ("presentac" in str(x) or "encuest" in str(x) or "publicida" in str(x) or "propagand" in str(x)) and ("tiempo" in str(x) or "perdida" in str(x) or "valioso" in str(x) or "minuto" in str(x) or " hora" in str(x)):
        return "CV5"
    elif ("diapositiva" in str(x) or "slide" in str(x) or "apuntes" in str(x) or "notas" in str(x) or "anotacion" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV7"
    elif ("formato" in str(x) or "encuesta" in str(x) or "llena" in str(x) or "relle" in str(x) or "completa" in str(x)) and ("concentra" in str(x) or "enfo" in str(x)):
        return "CV8" 
    elif ("informa" in str(x) or "avis" in str(x) or "comunic" in str(x) or "adver" in str(x) or "notific" in str(x) or "anunc" in str(x)) and ("participa" in str(x) or "entra" in str(x) or "acce" in str(x) or "meter" in str(x) or "incorpo" in str(x) or "estar" in str(x)) and ("clase" in str(x) or "modulo" in str(x) or "leccion" in str(x)):
        return "CV9"  
    elif ("confuso" in str(x) or "desordena" in str(x) or "enreda" in str(x) or "dific" in str(x) or "entende" in str(x) or "compli" in str(x) or "complej" in str(x)) and ("horario" in str(x) or " hora" in str(x) or "duracion" in str(x) or "horaria" in str(x) or "calendario" in str(x) or "crono" in str(x)):
        return "CV10"
    elif ("repe" in str(x) or "repiten" in str(x) or "reiter" in str(x) or "reincid" in str(x) or "vuelv" in str(x) or "rehace" in str(x) or " dos " in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV11"
    elif ("graba" in str(x) or "reprod" in str(x) or "guarda" in str(x) or "almac" in str(x) or "archiv" in str(x)) and ("acce" in str(x) or "entr" in str(x) or "ir" in str(x) or "mira" in str(x) or "entra" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV12"
    elif ("inscri" in str(x) or "anotar" in str(x) or "registr" in str(x) or "llenar" in str(x) or "escri" in str(x)) and ("clase" in str(x) or "vivo" in str(x)):
        return "CV13"
    elif ("no se ve" in str(x) or "ver" in str(x) or "mira" in str(x) or "observa" in str(x) or "borroso" in str(x) or "zoom" in str(x) or "entrecortado" in str(x)) and ("clase" in str(x) or "leccion" in str(x) or "lesion" in str(x) or "catedra" in str(x) or "modulo" in str(x)):
        return "CV14"
    elif ("tiempo" in str(x) or "periodo" in str(x) or "duracion" in str(x) or "durar" in str(x) or "lapso" in str(x) or "espacion" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "problema" in str(x) or "inquietud" in str(x) or "cuestion" in str(x)):
        return "CV15"
    elif ("grabada" in str(x) or "filmada" in str(x) or "repeti" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV16"
    elif ("repi" in str(x) or "repet" in str(x) or "reincid" in str(x) or "vuelven" in str(x) or "duplic" in str(x) or "rehac" in str(x) or "mismo" in str(x) or "diferente" in str(x) or "distinto" in str(x)) and ("pdf" in str(x)):
        return "CV17"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x) or "elemento" in str(x) or "pieza" in str(x) or "") and ("explica" in str(x) or "describ" in str(x) or "ensena" in str(x) or "justifi" in str(x)):
        return "CV18"
    elif ("practica" in str(x) or "ejercicio" in str(x) or "prueba" in str(x) or "ensay" in str(x) or "ejecu" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "vivo" in str(x)):
        return "CV19"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("economia" in str(x) or "ahorro" in str(x) or "ingreso" in str(x) or "dinero" in str(x) or "plata" in str(x) or "banco" in str(x) or "moneda" in str(x)):
        return "COV1"    
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "obtenido" in str(x) or "consegui" in str(x) or "encontra" in str(x) or "halla" in str(x) or "busca" in str(x)):
        return "COV2"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("enferm" in str(x) or "dolor" in str(x) or "duele" in str(x) or "muerto" in str(x) or "muerte" in str(x)):
        return "COV3"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("amplia" in str(x) or "prorroga" in str(x) or "aplaza" in str(x) or "suspen" in str(x) or "parar" in str(x) or "retras" in str(x)):
        return "COV4"   
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "DO1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x)):
        return "DO2"
    elif ("comunica" in str(x) or "habla" in str(x) or "conversa" in str(x) or "ver" in str(x) or "what" in str(x) or "chat" in str(x) or "telefo" in str(x) or "celula" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO3"
    elif ("varios" in str(x) or "diversos" in str(x) or "variado" in str(x) or "uno" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO4"
    elif ("llamada" in str(x) or "llama" in str(x) or "presenta" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO5"
    elif ("paciencia" in str(x) or "serenidad" in str(x) or "calma" in str(x) or "tranquilidad" in str(x) or "tranquilo" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "explica" in str(x) or "habla" in str(x)):
        return "DO6"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "replica" in str(x) or "comentario" in str(x) or "alimentacion" in str(x)) and ("actividad" in str(x) or "tarea" in str(x) or "practica" in str(x) or "trabajo" in str(x) or "resultado" in str(x) or "plato" in str(x) or "comida" in str(x)):
        return "DO8"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "comentario" in str(x) or "respond" in str(x) or "respue" in str(x)) and (("equivocad" in str(x) or "errado" in str(x)) or ("nombre" in str(x))):
        return "DO9"
    elif ("calificac" in str(x) or "nota" in str(x) or "puntuacion" in str(x)) and ("baja" in str(x) or "perdi" in str(x) or "perder" in str(x) or "exigente" in str(x) or "riguroso" in str(x)):
        return "DO10"
    elif ("profesional" in str(x) or "cualificado" in str(x) or "cualificado" in str(x) or "competente" in str(x)) and (("personal" in str(x) or "persona" in str(x) or "docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x))): #and ("sepa" in str(x) or "saber" in str(x) or "conocimiento" in str(x) or "tema" in str(x))):
        return "DO11"
    elif ("resolve" in str(x) or "resuel" in str(x) or "aclara" in str(x) or "respond" in str(x)) and ("receta" in str(x) or "practica" in str(x) or "ejercicio" in str(x)):
        return "DO12"
    elif ("retroalimenta" in str(x) or "feedback" in str(x) or "feedbac" in str(x) or "comentario" in str(x) or "resp" in str(x)) and ("video" in str(x)):
        return "DO13"
    elif ("enlace" in str(x) or "externo" in str(x) or "web" in str(x) or "internet" in str(x) or "pagina" in str(x) or "yout" in str(x) or "google" in str(x) or "gogle" in str(x)) and ("busca" in str(x) or "mirar" in str(x) or "investi" in str(x)):
        return "DO14"
    elif ("leer" in str(x) or "lee" in str(x) or "memori" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO15"
    elif ("nervio" in str(x) or "seguridad" in str(x) or "inquieto" in str(x) or "segur" in str(x) or "angustia" in str(x) or "tranqui" in str(x) or "histeric" in str(x) or "calma" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO16"
    elif ("habla" in str(x) or "conversa" in str(x) or "dialoga" in str(x) or "charla" in str(x) or "platica" in str(x) or "decir" in str(x) or "explica" in str(x) or "expresa" in str(x)) and ("lento" in str(x) or "claro" in str(x) or "pausado" in str(x) or "tranquil" in str(x) or "calma" in str(x)):
        return "DO17"
    elif ("viste" in str(x) or "presentacion personal" in str(x) or "elegan" in str(x) or "belleza" in str(x) or "ropa" in str(x) or "traje" in str(x) or "indumentaria" in str(x) or "prenda" in str(x) or "vestuario" in str(x) or "atuendo" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO18"
    elif ("retroaliment" in str(x)):
        return "DO20" #Revisar D20 y DO21 cuales son las diferencias
    elif (("promesa" in str(x) or "prometieron" in str(x) or "ofrecieron" in str(x) or "garantiza" in str(x) or "ofre" in str(x)) and ("vend" in str(x) or "mentira" in str(x) or "venta" in str(x) or "llamaron" in str(x) or "publicidad" in str(x) or "propaganda" in str(x)) or ("fraude" in str(x) or "engano" in str(x) or "estafa" in str(x) or "estafa" in str(x))):
        return "OT1"
    elif ("estoy" in str(x) or "siento" in str(x) or "encuentro" in str(x) or "senti" in str(x) or "tengo" in str(x)) and ("frustr" in str(x) or "trist" in str(x) or "fracas" in str(x) or "amarg" in str(x)):
        return "OT2"
    elif ("nada" in str(x) or "no" in str(x) or "cero" in str(x) or "ninguna" in str(x)) and ("comentario" in str(x) or "opinion" in str(x) or "juicio" in str(x) or "agregar" in str(x) or "decir" in str(x)) and ("posit" in str(x) or "buen" in str(x) or "agregar" in str(x) or "decir" in str(x)):
        return "OT3"
    elif ("motivacion" in str(x) or "motivado" in str(x) or "ganas" in str(x) or "estimulo" in str(x) or "animo" in str(x) or "aliento" in str(x) or "incentivo" in str(x)) and ("estudiar" in str(x) or "hacer l" in str(x) or "realizar l" or "aprender" in str(x) or "practic" in str(x) or "cocina" in str(x) or "medit" in str(x) or "entrar a" in str(x) or "iniciar el" in str(x) or "iniciar la" in str(x) or "empezar el" in str(x) or "empezar la" in str(x)):
        return "OT4"
    elif ("recibi" in str(x) or "envia" in str(x) or "mandado" in str(x) or "mandan" in str(x) or "despacha" in str(x) or "tramit" in str(x) or "dado" in str(x) or "dar" in str(x)) and ("diploma" in str(x) or "credencial" in str(x) or "certifica" in str(x) or "titulo" in str(x)):
        return "OT5"
    elif ("inici" in str(x) or "principio" in str(x) or "comienzo" in str(x) or "comenz" in str(x) or "comiens" in str(x)) and ("apoyo" in str(x) or "ayuda" in str(x) or "respaldo" in str(x) or "colabora" in str(x) or "duda" in str(x) or "pregunta" in str(x) or "inquietud" in str(x)):
        return "OT6"
    elif ("mensaj" in str(x) or "aviso" in str(x) or "whats" in str(x) or "llama" in str(x) or "correo" in str(x) or "celular" in str(x) or "telefono" in str(x)) and ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "plata" in str(x) or "dinero" in str(x)):
        return "OT7"
    elif ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "cobra" in str(x) or "plata" in str(x) or "dinero" in str(x)) and ("inscrito" in str(x) or "tomado" in str(x) or "adscrito" in str(x) or "tomando" in str(x) or "visto" in str(x) or "viendo" in str(x)):
        return "OT8"
    #elif (("soy" in str(x) or "siento" in str(x)) and ("talento" in str(x) or "aptitud" in str(x))) or (("soy malo" in str(x) or "soy bueno" in str(x) or "no tengo" in str(x) or "soy pesimo" in str(x) or "no puedo" in str(x))):
        #return "OT9" #mirar como el no tengo #MIRAR LOS NO TENGO
    elif ("interaccion" in str(x) or "interac" in str(x) or "hablar" in str(x) or "conocer" in str(x) or "charla" in str(x) or "platica" in str(x) or "ver" in str(x) or "videollamada" in str(x) or "reunion" in str(x)) and ("alumno" in str(x) or "estudiante" in str(x) or "companero" in str(x) or "pares" in str(x)):
        return "OT10"
    elif ("reconocimiento" in str(x)) and ("internacional" in str(x)):
        return "OT11"
    elif ("presencial" in str(x) or "fisico" in str(x) or "universidad" in str(x)):
        return "OT12"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x) or "manda" in str(x)) and ("video" in str(x) or "actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "archivo" in str(x)):
        return "PL1"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("navega" in str(x) or "explora" in str(x) or "oprimir" in str(x) or "web" in str(x) or "pagina" in str(x) or "entrar a" in str(x)) and not ("acceso" in str(x)):
        return "PL2"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x)) and ("grande" in str(x) or "pesa" in str(x) or "peso" in str(x)):
        return "PL3"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("acceso" in str(x) or "entrada" in str(x)):
        return "PL4"
    elif ("encuesta" in str(x)) and not ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL5"
    elif ("descarga" in str(x) or "bajar" in str(x) or "baja" in str(x) or "descargo" in str(x) or "download" in str(x) or "guarda" in str(x) or "almacen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "video" in str(x) or "archivo" in str(x)):
        return "PL6"
    elif ("encuesta" in str(x)) and ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL7"
    elif ("la app" in str(x) or "aplicacion" in str(x) or "moodle" in str(x)) or ("pagina" in str(x) and ("funcion" in str(x) or "sencil" in str(x) or "facil" in str(x) or "comod" in str(x))):
        return "PL8"
    elif ("avan" in str(x) or "progres" in str(x) or "aprobad" in str(x) or "pase" in str(x) or "evolucion" in str(x) or "cumplid" in str(x) or "realiz" in str(x) or "realis" in str(x) or "nota" in str(x) or "califica" in str(x)) and ("muestra" in str(x) or "regist" in str(x) or "exhibe" in str(x) or "present" in str(x) or "revela" in str(x) or "indica" in str(x) or "ver" in str(x) or "veo" in str(x) or "observa" in str(x)):
        return "PL9"
    elif ("funciona" in str(x) or "sirve" in str(x) or "desemp" in str(x) or "march" in str(x) or "traba" in str(x) or "trava" in str(x) or "estorba" in str(x) or "error" in str(x) or "falla" in str(x) or "fallo" in str(x) or "congela" in str(x) or "pasma" in str(x)) and ("material" in str(x) or "interac" in str(x) or "video" in str(x) or "tarjeta" in str(x) or "animado" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "animacion" in str(x) or "imagen" in str(x) or "grabacion" in str(x)):
        return "PL10"
    elif ("link" in str(x) or "enlace" in str(x) or "boton" in str(x) or "oprim" in str(x) or "hipervinculo" in str(x)) and ("descar" in str(x) or "bajar" in str(x) or "download" in str(x) or "materia" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "video" in str(x) or "receta" in str(x)) and not ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL11"
    elif ("clase" in str(x) or "sesion" in str(x)) and ("traba" in str(x) or "carga" in str(x) or "borra" in str(x) or "funciona" in str(x) or "escucha" in str(x) or "oye" in str(x) or "lenta" in str(x) or "lento" in str(x) or "congel" in str(x)):
        return "PL12"
    elif ("chat" in str(x)) or (("mensaje" in str(x)) and ("app" in str(x) or "aplicacion" in str(x))):
        return "PL13"
    elif ("notifica" in str(x) or "alerta" in str(x) or "señal" in str(x) or "aviso" in str(x)) and ("plataforma" in str(x) or "moodle" in str(x) or "app" in str(x) or "aplicacion" in str(x) or "pagina" in str(x) or "internet" in str(x)):
        return "PL14"
    elif ("reprod" in str(x) or "abri" in str(x) or "desple" in str(x) or "abre" in str(x) or "inic" in str(x) or "escuch" in str(x) or "oir" in str(x) or "ver" in str(x)) and ("video" in str(x) or "audio" in str(x) or "multimedia" in str(x) or "sonido" in str(x)) and ("dispositivo" in str(x) or "celular" in str(x) or "telefo" in str(x) or "computa" in str(x) or "pc" in str(x) or "equipo" in str(x) or "iphone" in str(x) or "android" in str(x) or "mac" in str(x) or "tablet" in str(x) or "pad" in str(x)):
        return "PL15"
    elif ("congela" in str(x) or " traba " in str(x) or "se pone lenta" in str(x) or "esta lenta" in str(x) or "esta muy lenta" in str(x) or "pantalla" in str(x) or "trava" in str(x) or "esta lento" in str(x) or "va lenta" in str(x)):
        return "PL16"
    elif ("enlace" in str(x) or "link" in str(x) or "hipervinculo" in str(x) or "boton" in str(x) or "oprim" in str(x)) and ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL17"  
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "SE1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x) or "atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("linea" in str(x) or "numero" in str(x) or "what" in str(x) or "facebook" in str(x) or "instagram" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)):
        return "SE2"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)) and ("otra area" in str(x) or "otro diplomado" in str(x) or "no sabe" in str(x) or "no tiene conocimiento" in str(x)):
        return "SE3"
    elif ("indu" in str(x)) and ("personal" in str(x) or "individual" in str(x) or "propio" in str(x) or "particular" in str(x) or "basico" in str(x) or "intermedio" in str(x) or "avanzado" in str(x) or "pc" in str(x) or "internet" in str(x) or "utilizar" in str(x) or "web" in str(x)):
        return "SE4"
    elif ("indu" in str(x)) and ("recibi" in str(x) or "dieron" in str(x) or "daron" in str(x) or "no pude" in str(x)):
        return "SE5"
    elif ("indu" in str(x)) and ("explica" in str(x) or "ensen" in str(x) or "aclara" in str(x) or "manifest" in str(x)):
        return "SE6"
    elif ("pacien" in str(x) or "seren" in str(x) or "tranquil" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("explica" in str(x) or "ensena" in str(x)):
        return "SE7"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "tutor" in str(x)) and ("contacto" in str(x) or "numero" in str(x) or "telefono" in str(x) or "info de contacto" in str(x) or "informacion de contacto" in str(x) or "informacion para contactarse" in str(x)):
        return "SE8"
    elif ("llamada" in str(x) or "mensaje" in str(x) or "llama" in str(x)) and ("mucha" in str(x) or "demasia" in str(x) or "cantidad" in str(x) or "exceso" in str(x) or "monton" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "dia" in str(x)):
        return "SE9"
    elif ("termine" in str(x) or "acab" in str(x) or "complet" in str(x) or "conclu" in str(x) or "finali" in str(x)) and ("modulo" in str(x) or "curso" in str(x) or "diplomado" in str(x) or "escuela" in str(x) or "especiali" in str(x) or "carrera" in str(x)):
        return "SE10"
    elif ("seguimiento" in str(x) or "atencion" in str(x) or "atend" in str(x) or "preocup" in str(x)) and ("mas" in str(x) or "mayor" in str(x) or "aument" in str(x)):
        return "SE11"  
    elif ("duda" in str(x) or "pregunta" in str(x) or "cuestion" in str(x)) and ("esfuer" in str(x) or "interes" in str(x) or "ganas" in str(x)) and not ("interesante" in str(x)):
        return "SE12"
    elif ("profundizar" in str(x)) and ("temas" in str(x) or "asunto" in str(x) or "cosa" in str(x) or "agenda" in str(x) or "inscribir" in str(x) or "usar" in str(x) or "uso" in str(x) or "agendar clase" in str(x) or "plataforma" in str(x) or "retroalimentacion" in str(x)):
        return "SE13"
    elif ("tiempo" in str(x) or "tarda" in str(x) or "demora" in str(x)) and ("llamada" in str(x) or "llama" in str(x) or "chat" in str(x) or "spar" in str(x) or "contesta" in str(x) or "resp" in str(x) or "ayuda" in str(x) or "apoyo" in str(x) or "acompa" in str(x)):
        return "SE14"     
    elif ("nunca" in str(x) or "jamas" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "respuesta" in str(x) or "colaboracion" in str(x) or "retroalimentacion" in str(x) or "contest" in str(x) or "llama" in str(x)):
        return "SE15"
    elif ("tiempo" in str(x) or "duracion" in str(x) or "dura" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)) and ("termina" in str(x) or "acaba" in str(x) or "culmin" in str(x) or "completa" in str(x) or "finali" or str(x) or "conclu" in str(x)):
        return "TD1"
    elif ("tiempo" in str(x) or "duracion" in str(x) or " hora" in str(x) or " dia" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("entrega" in str(x) or "subi" in str(x) or "envia" in str(x) or " manda" in str(x)):
        return "TD2"
    elif ("pausa" in str(x) or "congela" in str(x) or "parar" in str(x) or "vaciones" in str(x) or "enfermedad" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)):
        return "TD3"
    elif ("menos" in str(x) or "no mas" in str(x) or "muchas" in str(x) or "disminu" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR1"
    elif ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR2"
    elif ("sencill" in str(x) or "facil" in str(x) or "simple" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR3"
    elif ("tecnic" in str(x) or "teoric" in str(x) or "aplica" in str(x) or "videa real" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR4"
    elif ("dificil" in str(x) or "complica" in str(x) or "complej" in str(x) or "imposib" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR5"
    elif ("extens" in str(x) or "largo" in str(x) or "grande" in str(x) or "corto" in str(x) or "amplio" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR6"
    elif ("foto" in str(x) or "fotografia" in str(x) or "imagen" in str(x) or "prueba" in str(x)) and ("much" in str(x) or "demasi" in str(x) or "excesiv" in str(x) or "poco" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR7"
    elif ("pregunta" in str(x) or "piden" in str(x) or "pedir" in str(x) or "exig" in str(x) or "solici" in str(x)) and ("se vieron" in str(x) or "se abordaron" in str(x) or "no explic" in str(x) or "no ensen" in str(x)):
        return "PR8"
    elif ("instrucc" in str(x) or "paso" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("clar" in str(x) or "confus" in str(x)):
        return "PR9"
    elif ("indice" in str(x) or "clasificacion" in str(x) or "seguimiento" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR11"
    elif ("tiempo" in str(x) or "espacio" in str(x) or "periodo" in str(x) or "dias" in str(x) or " hora" in str(x)) and ("real" in str(x) or "coincid" in str(x) or "igual" in str(x) or "en verdad" in str(x)):
        return "PR12"
    elif ("formato" in str(x) or "amigable" in str(x) or "forma" in str(x) or "disen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR13"
    elif ("poder" in str(x) or "deberia" in str(x) or "seria" in str(x) or "opcion" in str(x) or "alternativ" in str(x) or "eleg" in str(x))  and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "audio" in str(x) or "grabac" in str(x)):
        return "PR14"
    elif ("examen" in str(x) or "ejercicio" in str(x) or "test" in str(x) or "evalua" in str(x)) or ("confus" in str(x) or "confun" in str(x)):
        return "EC1"
    elif ("respuesta bien" in str(x) or "respuesta esta mal" in str(x) or "respuesta no coincide" in str(x) or "respuesta mal" in str(x) or "respuesta bien" in str(x) or "marque la respuesta" in str(x) or "marco la respuesta" in str(x) or "marco la opcion" in str(x)):
        return "EC1"
    elif ("instruccio" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("entend" in str(x) or "entiend" in str(x) or "comprend" in str(x)):
        return "EC2"
    elif ("pregun" in str(x)) and ("no explic" in str(x) or "no die" or "no hi" or "no est"):
        return "EC3"
    elif ("preguntas son" in str(x) or "preguntas debe" in str(x) or "preguntas mas" in str(x) or "preguntas menos" in str(x)) and ("dificil" in str(x) or "facil" in str(x) or "comple" in str(x) or "teor" in str(x) or "tecnic" in str(x)):
        return "EC4"
    elif ("mas eva" in str(x) or "mas eje" in str(x) or "mas cues" in str(x)):
        return "EC5"
    elif ("retroal" in str(x)) and ("evalua" in str(x) or "cuestion" in str(x)):
        return "EC6"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "amplia" in str(x)):
        return "RE1" #mas miremos a ver si se quita
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("ingrediente" in str(x) or "alimento" in str(x) or "comida" in str(x) or "componente" in str(x)):
        return "RE2" #mirar porque está repetida ahí arribe(
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("utensilio" in str(x) or "util" in str(x) or "uten" in str(x) or "herramienta" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "oya" in str(x) or "sarten" in str(x) or "licuador" in str(x) or "cocina" in str(x) or "elemento" in str(x) or "tabla" in str(x) or "rallador" in str(x) or "pelador" in str(x) or "molde" in str(x) or "cacerola" in str(x)):
        return "RE3"
    elif (" tips" in str(x) or "truco" in str(x) or "atajo" in str(x) or "consejo" in str(x) or "recomenda" in str(x) or "sugerenci" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE5"
    elif ("actual" in str(x) or " nuev" in str(x) or " tenden" in str(x) or "presente" in str(x) or "vigente" in str(x) or "moda" in str(x) or "ultimas" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE6"
    elif ("porq" in str(x) or "por q" in str(x) or "motivo" in str(x) or "razon" in str(x) or "motivacion" in str(x) or "fundamento" in str(x) or "finalidad" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE7"
    elif ("receta" in str(x) or "formula" in str(x)) and ("especif" in str(x) or "distint" in str(x) or "especial" in str(x) or "particu" in str(x) or "diferen" in str(x) or "paso a paso" in str(x)):
        return "RE11"
    elif ("gramo" in str(x) or " tasa " in str(x) or " taza " in str(x) or "onza" in str(x) or " onsa" in str(x) or "kilo" in str(x) or "libra" in str(x)):
        return "RE9"
    elif ("receta" in str(x) or "formula" in str(x)) and ("descar" in str(x) or " baja" in str(x)):
        return "RE10"
    elif ("receta" in str(x) or "formula" in str(x)) and ("igual" in str(x) or "coinci" in str(x) or "concord" in str(x) or "similar" in str(x) or "equival" in str(x) or "mismo" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "clase" in str(x)):
        return "RE12"
    elif ("paso a paso" in str(x) or "congruen" in str(x) or "cohere" in str(x)) and ("recet" in str(x) or "formula" in str(x)):
        return "RE13"
    elif ("tamano" in str(x) or "medida" in str(x) or "dimensio" in str(x) or "talla" in str(x) or "grande" in str(x) or "peque" in str(x)) and ("molde" in str(x) or "olla" in str(x) or "cacerola" in str(x) or "cuchillo" in str(x) or "tabla" in str(x) or "rallado" in str(x)):
        return "RE14"
    elif ("receta" in str(x)) and ("rubric" in str(x)):
        return "RE15"
    elif ("receta" in str(x) or "formula" in str(x)) and ("teoria" in str(x)):
        return "RE16"
    elif ("receta" in str(x) or "formula" in str(x) or "plato" in str(x) or "comida" in str(x) or "postre" in str(x)) and ("tiempo" in str(x) or "minuto" in str(x) or "hora" in str(x) or "duracion" in str(x) or "segundo" in str(x)):
        return "RE17"
    elif ("receta" in str(x) or "plato" in str(x) or "plato" in str(x) or "postre" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "pastel" in str(x) or "parrillada" in str(x) or "asado" in str(x)) and ("resultado" in str(x) or "final" in str(x) or "producto" in str(x) or "feo" in str(x) or "sabor" in str(x) or "quedo mal" in str(x) or "se ve mal" in str(x) or " sabe" in str(x) or "se ve feo" in str(x) or "no sale" in str(x) or "no sirve" in str(x)):
        return "RE19"
    elif ("receta" in str(x) or "formula" in str(x)) and ("diseno" in str(x) or "fotografia" in str(x) or " foto" in str(x) or " imagen" in str(x)):
        return "RE20"
    elif ("utensilio" in str(x) or "instrumento" in str(x) or "herramienta" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "molde" in str(x) or "rallador" in str(x) or "tijera" in str(x) or "olla" in str(x) or "batidor" in str(x) or "vaso" in str(x)) and ("explica" in str(x) or "ilustr" in str(x) or "ensen" in str(x) or "indica" in str(x) or "da clases" in str(x) or "educar" in str(x)):
        return "RE21"
    elif ("receta" in str(x) or "formula" in str(x) or "alimento" in str(x) or "comida" in str(x) or "bandeja" in str(x) or "parrill" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "arroz" in str(x) or "postre" in str(x) or "torta" in str(x)) and ("difici" in str(x) or "complej" in str(x) or "complica" in str(x) or "imposibl" in str(x) or "arduo" in str(x) or "fácil" in str(x) or "sencill" in str(x) or "simple" in str(x)):
        return "RE22"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovisu" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and not ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)) :
        return "VI1"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)):
        return "VI2"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("pasos" in str(x) or "proceso" in str(x)):
        return "VI3"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("texto" in str(x) or "escrito" in str(x)):
        return "VI4"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("interact" in str(x)):
        return "VI5"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("ia" in str(x) or "tema" in str(x) or "topic" in str(x) or "materia" in str(x)):
        return "VI6"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mismo" in str(x) or "misma" in str(x) or "ident" in str(x) or "igual" in str(x) or "equivalente" in str(x)):
        return "VI7"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x) or "imagen" in str(x) or "foto" in str(x)) and (" mas " in str(x)):
        return "VI8"
    elif ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)):
        return "VI9"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("calidad" in str(x) or "borroso" in str(x) or "no se ve bien" in str(x) or "imagen" in str(x) or "audio" in str(x) or "volumen" in str(x) or "sonido" in str(x) or "oscuro" in str(x) or "gris" in str(x)):
        return "VI11"
    elif ("set" in str(x) or " estudio" in str(x) or "cocina" in str(x) or "lugar" in str(x) or "sitio" in str(x)) and ("limpio" in str(x) or "aseado" in str(x) or "pulcro" in str(x) or "higienico" in str(x) or "sucio" in str(x)):
        return "VI12"
    elif ("video" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "VI13"
    elif ("video" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x)):
        return "VI14"
    elif ("video" in str(x)) and ("experiencia" in str(x) or "truco" in str(x) or "consejo" in str(x) or "sugerencia" in str(x) or "real" in str(x) or "solucion" in str(x) or "reparar" in str(x)):
        return "VI15"
    elif ("subtitulo" in str(x)):
        return "VI16"
    elif ("video" in str(x)) and ("actual" in str(x) or "reciente" in str(x)):
        return "VI17"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "AU1"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x) or "dura" in str(x)):
        return "AU2"
    elif ("audio" in str(x) or "sonido" in str(x) or "volumen" in str(x)) and ("calidad" in str(x) or "volumen" in str(x) or "escuch" in str(x) or "bajo" in str(x) or "alto" in str(x) or "debil" in str(x) or "ruido" in str(x)):
        return "AU3"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "poco" in str(x) or "increment" in str(x)):
        return "AU5"
    elif("audio" in str(x) or "sonido" in str(x) or "ruido" in str(x)):
        return "AU4"
    else:
        return "-"

In [24]:
def clasificador2_2(x):
    if ("fenomenal" in str(x) or "todo super" in str(x) or "me parece bien" in str(x) or "me parece perfecto" in str(x) or "muy bueno" in str(x) or "muy buena" in str(x) or "muy bien" in str(x) or "esta bien" in str(x) or "felicito" in str(x) or "felicidades" in str(x) or "felicitaciones" in str(x) or "muy util" in str(x) or "muy interesante" in str(x) or "siga igual" in str(x) or "sigan igual" in str(x) or "va bien" in str(x) or "muy bien" in str(x) or "todo va bien" in str(x) or "genial" in str(x) or "encanto" in str(x) or "encantar" in str(x) or "perfecto" in str(x) or "todo bien" in str(x) or "todo muy bien" in str(x) or "exc" in str(x) or "satisfech" in str(x) or "satisfacc" in str(x) or "me gusta " in str(x) or "me gusto" in str(x) or "todo bien" in str(x) or "sigan asi" in str(x) or "maravilloso" in str(x) or "nada que mej" in str(x) or "maravilla" in str(x) or "gracias por" in str(x) or "gracias a" in str(x) or "muchas gracias" in str(x) or "muchas grasias" in str(x) or "feliz" in str(x) or "contento" in str(x)) and not ("no " in str(x) or "video" in str(x) or "imagen" in str(x) or "foto" in str(x)):
        return "NO"
    elif ("ninguno" in str(x) or "ninguna" in str(x) or "n/a" in str(x) or "sin com" in str(x) or "sin sug" in str(x) or "sin opi" in str(x) or "ningun come" in str(x) or "ningun sug" in str(x) or "        " in str(x) or "nada que agregar" in str(x) or "ningun comentario" in str(x) or "nada que decir" in str(x) or "no tengo" in str(x)) and not ("descarga" in str(x) or "docente" in str(x) or "video" in str(x)):
        return "NO"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovisu" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and not ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)) :
        return "VI1"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("receta" in str(x) or "plato" in str(x) or "comida" in str(x)) and ("detall" in str(x) or "textur" in str(x) or "consis" in str(x) or "minucios" in str(x) or "precis" in str(x)):
        return "VI2"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("pasos" in str(x) or "proceso" in str(x)):
        return "VI3"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("texto" in str(x) or "escrito" in str(x)):
        return "VI4"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("interact" in str(x)):
        return "VI5"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "mucho" in str(x) or "poco" in str(x) or "increment" in str(x)) and ("tema" in str(x) or "topic" in str(x) or "materia" in str(x)):
        return "VI6"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("mismo" in str(x) or "misma" in str(x) or "ident" in str(x) or "igual" in str(x) or "equivalente" in str(x)):
        return "VI7"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("imagen" in str(x) or "foto" in str(x) or "ilustra" in str(x)) and (" mas " in str(x)):
        return "VI8"
    elif ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)):
        return "VI9"
    elif ("video" in str(x) or "grabac" in str(x) or "grabar" in str(x) or "audiovis" in str(x)) and ("calidad" in str(x) or "borroso" in str(x) or "no se ve bien" in str(x) or "imagen" in str(x) or "audio" in str(x) or "volumen" in str(x) or "sonido" in str(x) or "oscuro" in str(x) or "gris" in str(x)):
        return "VI11"
    elif ("set" in str(x) or " estudio" in str(x) or "cocina" in str(x) or "lugar" in str(x) or "sitio" in str(x)) and ("limpio" in str(x) or "aseado" in str(x) or "pulcro" in str(x) or "higienico" in str(x) or "sucio" in str(x)):
        return "VI12"
    elif ("video" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "VI13"
    elif ("video" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x)):
        return "VI14"
    elif ("video" in str(x)) and ("experiencia" in str(x) or "truco" in str(x) or "consejo" in str(x) or "sugerencia" in str(x) or "real" in str(x) or "solucion" in str(x) or "reparar" in str(x)):
        return "VI15"
    elif ("subtitulo" in str(x)):
        return "VI16"
    elif ("video" in str(x)) and ("actual" in str(x) or "reciente" in str(x)):
        return "VI17"
    elif("video" in str(x)) and ("mas " in str(x)):
        return "VI4"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("largo" in str(x) or "extenso" in str(x) or "amplio" in str(x) or " dura " in str(x)):
        return "AU1"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("corto" in str(x) or "pequeno" in str(x) or "consis" in str(x) or "tiempo" in str(x) or "duracion" in str(x) or "conci" in str(x) or "dura" in str(x)):
        return "AU2"
    elif ("audio" in str(x) or "sonido" in str(x) or "volumen" in str(x)) and ("calidad" in str(x) or "volumen" in str(x) or "escuch" in str(x) or "bajo" in str(x) or "alto" in str(x) or "debil" in str(x) or "ruido" in str(x)):
        return "AU3"
    elif ("audio" in str(x) or "sonido" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "poco" in str(x) or "increment" in str(x)):
        return "AU5"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x)):
        return "DO2"
    elif("audio" in str(x) or "sonido" in str(x) or "ruido" in str(x)):
        return "AU4"
    elif ("descarga" in str(x) or "bajar" in str(x) or "baja" in str(x) or "descargo" in str(x) or "download" in str(x) or "guarda" in str(x) or "almacen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "video" in str(x) or "archivo" in str(x)):
        return "PL6"
    elif ("detall" in str(x) or "precis" in str(x) or "especifi" in str(x) or "abarc" in str(x) or "aclara" in str(x) or "puntua" in str(x) or "fondo" in str(x) or "profund" in str(x) or "concre" in str(x) or "especi" in str(x)) and ("lectu" in str(x) or "conteni" in str(x) or "tema" in str(x) or "receta" in str(x) or "formula" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "info" in str(x) or "document" in str(x)):
        return "CO1"
    elif ("exten" in str(x) or "ampli" in str(x) or "grande" in str(x) or "reduci" in str(x) or "corto" in str(x) or "disminu" in str(x) or "largo" in str(x) or "breve" in str(x) or "conciso" in str(x) or "dismin" in str(x) or "satura" in str(x) or "tedio" in str(x)) and ("lectura" in str(x) or "conteni" in str(x) or "tema" in str(x) or "info" in str(x) or "leccion" in str(x) or "pdf" in str(x) or "document" in str(x) or "recet" in str(x)):
        return "CO2"
    elif (" mas " in str(x) or "mayor" in str(x) or "aument" in str(x) or "crec" in str(x) or "increment" in str(x) or "ampli" in str(x) or "anad" in str(x) or "agreg" in str(x) or "poco" in str(x) or "peque" in str(x) or "falt" in str(x) or "limit" in str(x)) and ("material" in str(x) or "libro" in str(x) or "herramienta" in str(x) or "util" in str(x) or "lectura" in str(x)):
        return "CO3"
    elif ("didactico" in str(x) or "pedago" in str(x) or "educativo" in str(x) or "interact" in str(x) or "imagene" in str(x) or "fotos" in str(x) or "dinamico" in str(x)) and not ("subi" in str(x) or "descargar" in str(x) or "envia" in str(x) or "baja" in str(x)):
        return "CO4"
    elif ("descarg" in str(x) or " baja" in str(x) or "consulta" in str(x)) and ("contenid" in str(x) or "video" in str(x) or "audio" in str(x) or "recurso" in str(x) or "pdf" in str(x) or "book" in str(x) or "recet" in str(x) or "plantilla" in str(x) or "material" in str(x)) and ("mas" in str(x) or "tener" in str(x) or "todo" in str(x)):
        return "CO5" #mirar
    elif ("sustitu" in str(x) or "opcion" in str(x) or "reempl" in str(x) or "supli" in str(x) or "relevar" in str(x) or "cambi" in str(x)) and ("material" in str(x) or "herramienta" in str(x) or "insumo" in str(x) or "componen" in str(x) or "produc" in str(x)):
        return "CO6"
    elif ("idioma" in str(x) or "ingles" in str(x) or "portugues" in str(x) or "espanol" in str(x) or "castellano" in str(x)):
        return "CO8"
    elif ("repite" in str(x) or "mismo" in str(x) or "repeticion" in str(x) or "aporta" in str(x) or "sirve" in str(x) or "diferente" in str(x) or "igual" in str(x) or "identico" in str(x) or "similar" in str(x)) and ("pdf" in str(x)):
        return "CO9"
    elif ("redaccion" in str(x) or "puntuacion" in str(x) or "puntuacion" in str(x) or "ortogra" in str(x) or "redac" in str(x)) or (("error" in str(x) or "equivo" in str(x)) and "escri" in str(x)):
        return "CO10"
    elif ("informacion" in str(x) or "texto" in str(x) or "pdf" in str(x) or "material" in str(x)) and ("incompleto" in str(x) or "confuso" in str(x) or "incongruente" in str(x) or "contradictorio" in str(x) or "repetid" in str(x) or "claro" in str(x) or "complet" in str(x) or "comprende" in str(x)):
        return "CO11"
    elif ("glosario" in str(x) or "vocabulario" in str(x)) or ("palabra" in str(x) and "clave" in str(x)):
        return "CO12"
    elif ("ejemplos" in str(x) or "ejercicio" in str(x) or "ejercicios" in str(x) or "ejemplo" in str(x) or "practica" in str(x)) and ("real" in str(x) or "aplicar" in str(x) or "aplicado" in str(x) or "vida" in str(x)):
        return "CO13"
    elif ("inicia" in str(x) or "empezar" in str(x) or "principio" in str(x) or "inicio" in str(x) or "primero" in str(x)) and ("lista" in str(x) or "herramientas" in str(x) or "ingredientes" in str(x) or "equipo" in str(x) or "catalogo" in str(x) or "inventario" in str(x)):
        return "CO14"
    elif ("actuali" in str(x) or "renova" in str(x) or "moderniz" in str(x) or "nuev" in str(x) or "reciente" in str(x) or "tendencia" in str(x)) and ("info" in str(x) or "texto" in str(x) or "tema" in str(x) or "leccion" in str(x) or "asunto" in str(x)):
        return "CO15"
    elif ("autoria" in str(x) or "bibliografia" in str(x) or "inventor" in str(x)):
        return "CO16"
    elif ("bibliografia" in str(x) or "texto" in str(x) or "video" in str(x) or "libro" in str(x) or "pagina" in str(x) or "recurso" in str(x) or "web" in str(x)) and ("profundiz" in str(x) or "reflex" in str(x) or "estudi" in str(x) or "indag" in str(x) or "investi" in str(x)):
        return "CO17"
    elif ("resum" in str(x) or "sinte" in str(x) or "sumario" in str(x)) and ("vist" in str(x) or "curso" in str(x) or "modulo" in str(x) or "leccion" in str(x) or "diplomado" in str(x)):
        return "CO18"
    elif ("contenido" in str(x) or "tema" in str(x) or "materia" in str(x)) and ("mas" in str(x) or "aument" in str(x) or "crecim" in str(x) or "anadir" in str(x) or "pusi" in str(x)) and ("ia" in str(x)):
        return "CO19"
    elif ("diseno" in str(x) or "calidad" in str(x) or "bonito" in str(x) or "lindo" in str(x) or "feo" in str(x) or "color" in str(x) or "tipografia" in str(x)) and ("imagen" in str(x) or "foto" in str(x) or "visual" in str(x) or "video" in str(x) or "letra" in str(x) or "fuente" in str(x) or "palabra" in str(x)):
        return "CO21"
    elif ("lenguaje" in str(x) or "idioma" in str(x) or "expresa" in str(x)) and ("dificil" in str(x) or "enredado" in str(x) or "complica" in str(x) or "compren" in str(x) or "entend" in str(x) or "entien" in str(x) or "facil" in str(x) or "senci" in str(x)):
        return "CO22" #mirar
    elif ("mismo" in str(x) or "igual" in str(x)) and ("plataforma" in str(x) or "internet" in str(x) or "fuentes" in str(x) or "pagar" in str(x) or "libro" in str(x) or "youtube" in str(x) or "yutube" in str(x)):
        return "CO23"
    elif ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "conseguir" in str(x) or "buscar" in str(x)) and ("insumos" in str(x) or "material" in str(x) or "herramienta" in str(x) or "utensilio" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "receta" in str(x) or "carne" in str(x) or "azuca" in str(x) or "ingredie" in str(x) or "olla" in str(x) or "cuchillo" in str(x) or "pollo" in str(x)):
        return "CO24"
    elif ("pais" in str(x) or "ciudad" in str(x)):
        return "CO25"    
    elif ("frecuencia" in str(x) or "frecue" in str(x) or "habitua" in str(x) or "corriente" in str(x) or "repetir" in str(x) or "usual" in str(x) or "numero" in str(x) or "cantidad" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV1"
    elif (" hora" in str(x) or "jornada" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "calendario" in str(x) or "flex" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV2"
    elif ("interac" in str(x) or "interrelac" in str(x) or "relacio" in str(x) or "comunicar" in str(x) or "trato" in str(x) or "intercambio" in str(x) or "saludo" in str(x) or "conocer" in str(x)) and ("alumno" in str(x) or "docente" in str(x) or "estudiante" in str(x) or "profesor" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "catedratico" in str(x) or "chef" in str(x)):
        return "CV3"
    elif ("duda" in str(x) or "pregunt" in str(x) or "aclara" in str(x) or "entien" in str(x) or "enten" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x) or "vivo" in str(x)):
        return "CV4"
    elif ("presentac" in str(x) or "encuest" in str(x) or "publicida" in str(x) or "propagand" in str(x)) and ("tiempo" in str(x) or "perdida" in str(x) or "valioso" in str(x) or "minuto" in str(x) or " hora" in str(x)):
        return "CV5"
    elif ("diapositiva" in str(x) or "slide" in str(x) or "apuntes" in str(x) or "notas" in str(x) or "anotacion" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "catedra" in str(x) or "leccion" in str(x)):
        return "CV7"
    elif ("formato" in str(x) or "encuesta" in str(x) or "llena" in str(x) or "relle" in str(x) or "completa" in str(x)) and ("concentra" in str(x) or "enfo" in str(x)):
        return "CV8" 
    elif ("informa" in str(x) or "avis" in str(x) or "comunic" in str(x) or "adver" in str(x) or "notific" in str(x) or "anunc" in str(x)) and ("participa" in str(x) or "entra" in str(x) or "acce" in str(x) or "meter" in str(x) or "incorpo" in str(x) or "estar" in str(x)) and ("clase" in str(x) or "modulo" in str(x) or "leccion" in str(x)):
        return "CV9"  
    elif ("confuso" in str(x) or "desordena" in str(x) or "enreda" in str(x) or "dific" in str(x) or "entende" in str(x) or "compli" in str(x) or "complej" in str(x)) and ("horario" in str(x) or " hora" in str(x) or "duracion" in str(x) or "horaria" in str(x) or "calendario" in str(x) or "crono" in str(x)):
        return "CV10"
    elif ("repe" in str(x) or "repiten" in str(x) or "reiter" in str(x) or "reincid" in str(x) or "vuelv" in str(x) or "rehace" in str(x) or " dos " in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV11"
    elif ("graba" in str(x) or "reprod" in str(x) or "guarda" in str(x) or "almac" in str(x) or "archiv" in str(x)) and ("acce" in str(x) or "entr" in str(x) or "ir" in str(x) or "mira" in str(x) or "entra" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV12"
    elif ("inscri" in str(x) or "anotar" in str(x) or "registr" in str(x) or "llenar" in str(x) or "escri" in str(x)) and ("clase" in str(x) or "vivo" in str(x)):
        return "CV13"
    elif ("no se ve" in str(x) or "ver" in str(x) or "mira" in str(x) or "observa" in str(x) or "borroso" in str(x) or "zoom" in str(x) or "entrecortado" in str(x)) and ("clase" in str(x) or "leccion" in str(x) or "lesion" in str(x) or "catedra" in str(x) or "modulo" in str(x)):
        return "CV14"
    elif ("tiempo" in str(x) or "periodo" in str(x) or "duracion" in str(x) or "durar" in str(x) or "lapso" in str(x) or "espacion" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "problema" in str(x) or "inquietud" in str(x) or "cuestion" in str(x)):
        return "CV15"
    elif ("grabada" in str(x) or "filmada" in str(x) or "repeti" in str(x)) and ("clase" in str(x) or "modulo" in str(x)):
        return "CV16"
    elif ("repi" in str(x) or "repet" in str(x) or "reincid" in str(x) or "vuelven" in str(x) or "duplic" in str(x) or "rehac" in str(x) or "mismo" in str(x) or "diferente" in str(x) or "distinto" in str(x)) and ("pdf" in str(x)):
        return "CV17"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x) or "elemento" in str(x) or "pieza" in str(x) or "") and ("explica" in str(x) or "describ" in str(x) or "ensena" in str(x) or "justifi" in str(x)):
        return "CV18"
    elif ("practica" in str(x) or "ejercicio" in str(x) or "prueba" in str(x) or "ensay" in str(x) or "ejecu" in str(x)) and ("modulo" in str(x) or "clase" in str(x) or "vivo" in str(x)):
        return "CV19"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("economia" in str(x) or "ahorro" in str(x) or "ingreso" in str(x) or "dinero" in str(x) or "plata" in str(x) or "banco" in str(x) or "moneda" in str(x)):
        return "COV1"    
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("comprar" in str(x) or "adquirir" in str(x) or "obtener" in str(x) or "obtenido" in str(x) or "consegui" in str(x) or "encontra" in str(x) or "halla" in str(x) or "busca" in str(x)):
        return "COV2"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("enferm" in str(x) or "dolor" in str(x) or "duele" in str(x) or "muerto" in str(x) or "muerte" in str(x)):
        return "COV3"
    elif ("covid" in str(x) or "19" in str(x) or "covi" in str(x) or "pandemia" in str(x) or "salud" in str(x) or "cuarentena" in str(x)) and ("amplia" in str(x) or "prorroga" in str(x) or "aplaza" in str(x) or "suspen" in str(x) or "parar" in str(x) or "retras" in str(x)):
        return "COV4"   
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "DO1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x)):
        return "DO2"
    elif ("comunica" in str(x) or "habla" in str(x) or "conversa" in str(x) or "ver" in str(x) or "what" in str(x) or "chat" in str(x) or "telefo" in str(x) or "celula" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO3"
    elif ("varios" in str(x) or "diversos" in str(x) or "variado" in str(x) or "uno" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO4"
    elif ("llamada" in str(x) or "llama" in str(x) or "presenta" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO5"
    elif ("paciencia" in str(x) or "serenidad" in str(x) or "calma" in str(x) or "tranquilidad" in str(x) or "tranquilo" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "explica" in str(x) or "habla" in str(x)):
        return "DO6"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "replica" in str(x) or "comentario" in str(x) or "alimentacion" in str(x)) and ("actividad" in str(x) or "tarea" in str(x) or "practica" in str(x) or "trabajo" in str(x) or "resultado" in str(x) or "plato" in str(x) or "comida" in str(x)):
        return "DO8"
    elif ("retroalimentacion" in str(x) or "feedback" in str(x) or "opinion" in str(x) or "comentario" in str(x) or "respond" in str(x) or "respue" in str(x)) and (("equivocad" in str(x) or "errado" in str(x)) or ("nombre" in str(x))):
        return "DO9"
    elif ("calificac" in str(x) or "nota" in str(x) or "puntuacion" in str(x)) and ("baja" in str(x) or "perdi" in str(x) or "perder" in str(x) or "exigente" in str(x) or "riguroso" in str(x)):
        return "DO10"
    elif ("profesional" in str(x) or "cualificado" in str(x) or "cualificado" in str(x) or "competente" in str(x)) and (("personal" in str(x) or "persona" in str(x) or "docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x))): #and ("sepa" in str(x) or "saber" in str(x) or "conocimiento" in str(x) or "tema" in str(x))):
        return "DO11"
    elif ("resolve" in str(x) or "resuel" in str(x) or "aclara" in str(x) or "respond" in str(x)) and ("receta" in str(x) or "practica" in str(x) or "ejercicio" in str(x)):
        return "DO12"
    elif ("retroalimenta" in str(x) or "feedback" in str(x) or "feedbac" in str(x) or "comentario" in str(x) or "resp" in str(x)) and ("video" in str(x)):
        return "DO13"
    elif ("enlace" in str(x) or "externo" in str(x) or "web" in str(x) or "internet" in str(x) or "pagina" in str(x) or "yout" in str(x) or "google" in str(x) or "gogle" in str(x)) and ("busca" in str(x) or "mirar" in str(x) or "investi" in str(x)):
        return "DO14"
    elif ("leer" in str(x) or "lee" in str(x) or "memori" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO15"
    elif ("nervio" in str(x) or "seguridad" in str(x) or "inquieto" in str(x) or "segur" in str(x) or "angustia" in str(x) or "tranqui" in str(x) or "histeric" in str(x) or "calma" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO16"
    elif ("habla" in str(x) or "conversa" in str(x) or "dialoga" in str(x) or "charla" in str(x) or "platica" in str(x) or "decir" in str(x) or "explica" in str(x) or "expresa" in str(x)) and ("lento" in str(x) or "claro" in str(x) or "pausado" in str(x) or "tranquil" in str(x) or "calma" in str(x)):
        return "DO17"
    elif ("viste" in str(x) or "presentacion personal" in str(x) or "elegan" in str(x) or "belleza" in str(x) or "ropa" in str(x) or "traje" in str(x) or "indumentaria" in str(x) or "prenda" in str(x) or "vestuario" in str(x) or "atuendo" in str(x)) and ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x)):
        return "DO18"
    elif ("retroaliment" in str(x)):
        return "DO20" #Revisar D20 y DO21 cuales son las diferencias
    elif (("promesa" in str(x) or "prometieron" in str(x) or "ofrecieron" in str(x) or "garantiza" in str(x) or "ofre" in str(x)) and ("vend" in str(x) or "mentira" in str(x) or "venta" in str(x) or "llamaron" in str(x) or "publicidad" in str(x) or "propaganda" in str(x)) or ("fraude" in str(x) or "engano" in str(x) or "estafa" in str(x) or "estafa" in str(x))):
        return "OT1"
    elif ("estoy" in str(x) or "siento" in str(x) or "encuentro" in str(x) or "senti" in str(x) or "tengo" in str(x)) and ("frustr" in str(x) or "trist" in str(x) or "fracas" in str(x) or "amarg" in str(x)):
        return "OT2"
    elif ("nada" in str(x) or "no" in str(x) or "cero" in str(x) or "ninguna" in str(x)) and ("comentario" in str(x) or "opinion" in str(x) or "juicio" in str(x) or "agregar" in str(x) or "decir" in str(x)) and ("posit" in str(x) or "buen" in str(x) or "agregar" in str(x) or "decir" in str(x)):
        return "OT3"
    elif ("motivacion" in str(x) or "motivado" in str(x) or "ganas" in str(x) or "estimulo" in str(x) or "animo" in str(x) or "aliento" in str(x) or "incentivo" in str(x)) and ("estudiar" in str(x) or "hacer l" in str(x) or "realizar l" or "aprender" in str(x) or "practic" in str(x) or "cocina" in str(x) or "medit" in str(x) or "entrar a" in str(x) or "iniciar el" in str(x) or "iniciar la" in str(x) or "empezar el" in str(x) or "empezar la" in str(x)):
        return "OT4"
    elif ("recibi" in str(x) or "envia" in str(x) or "mandado" in str(x) or "mandan" in str(x) or "despacha" in str(x) or "tramit" in str(x) or "dado" in str(x) or "dar" in str(x)) and ("diploma" in str(x) or "credencial" in str(x) or "certifica" in str(x) or "titulo" in str(x)):
        return "OT5"
    elif ("inici" in str(x) or "principio" in str(x) or "comienzo" in str(x) or "comenz" in str(x) or "comiens" in str(x)) and ("apoyo" in str(x) or "ayuda" in str(x) or "respaldo" in str(x) or "colabora" in str(x) or "duda" in str(x) or "pregunta" in str(x) or "inquietud" in str(x)):
        return "OT6"
    elif ("mensaj" in str(x) or "aviso" in str(x) or "whats" in str(x) or "llama" in str(x) or "correo" in str(x) or "celular" in str(x) or "telefono" in str(x)) and ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "plata" in str(x) or "dinero" in str(x)):
        return "OT7"
    elif ("pagar" in str(x) or "pague" in str(x) or "abonar" in str(x) or "desembolsar" in str(x) or "cobrar" in str(x) or "pago" in str(x) or "cobra" in str(x) or "plata" in str(x) or "dinero" in str(x)) and ("inscrito" in str(x) or "tomado" in str(x) or "adscrito" in str(x) or "tomando" in str(x) or "visto" in str(x) or "viendo" in str(x)):
        return "OT8"
    #elif (("soy" in str(x) or "siento" in str(x)) and ("talento" in str(x) or "aptitud" in str(x))) or (("soy malo" in str(x) or "soy bueno" in str(x) or "no tengo" in str(x) or "soy pesimo" in str(x) or "no puedo" in str(x))):
        #return "OT9" #mirar como el no tengo #MIRAR LOS NO TENGO
    elif ("interaccion" in str(x) or "interac" in str(x) or "hablar" in str(x) or "conocer" in str(x) or "charla" in str(x) or "platica" in str(x) or "ver" in str(x) or "videollamada" in str(x) or "reunion" in str(x)) and ("alumno" in str(x) or "estudiante" in str(x) or "companero" in str(x) or "pares" in str(x)):
        return "OT10"
    elif ("reconocimiento" in str(x)) and ("internacional" in str(x)):
        return "OT11"
    elif ("presencial" in str(x) or "fisico" in str(x) or "universidad" in str(x)):
        return "OT12"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x) or "manda" in str(x)) and ("video" in str(x) or "actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "archivo" in str(x)):
        return "PL1"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("navega" in str(x) or "explora" in str(x) or "oprimir" in str(x) or "web" in str(x) or "pagina" in str(x) or "entrar a" in str(x)) and not ("acceso" in str(x)):
        return "PL2"
    elif ("subir" in str(x) or "enviar" in str(x) or "subo" in str(x) or "envio" in str(x) or "envie" in str(x) or "carga" in str(x) or "envia" in str(x)) and ("grande" in str(x) or "pesa" in str(x) or "peso" in str(x)):
        return "PL3"
    elif ("dificil" in str(x) or "complicado" in str(x) or "arduo" in str(x) or "complej" in str(x) or "imposible" in str(x) or "enreda" in str(x) or "dificul" in str(x) or "agota" in str(x)) and ("acceso" in str(x) or "entrada" in str(x)):
        return "PL4"
    elif ("encuesta" in str(x)) and not ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL5"
    elif ("descarga" in str(x) or "bajar" in str(x) or "baja" in str(x) or "descargo" in str(x) or "download" in str(x) or "guarda" in str(x) or "almacen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "evaluacion" in str(x) or "cuestionario" in str(x) or "evalua" in str(x) or "tarea" in str(x) or "ejercicio" in str(x) or "foto" in str(x) or "imagen" in str(x) or "receta" in str(x) or "pdf" in str(x) or "word" in str(x) or "excel" in str(x) or "fotografia" in str(x) or "plato" in str(x) or "documento" in str(x) or "video" in str(x) or "archivo" in str(x)):
        return "PL6"
    elif ("encuesta" in str(x)) and ("mucha" in str(x) or "poca" in str(x) or "demasiada" in str(x) or "cantidad" in str(x)):
        return "PL7"
    elif ("la app" in str(x) or "aplicacion" in str(x) or "moodle" in str(x)) or ("pagina" in str(x) and ("funcion" in str(x) or "sencil" in str(x) or "facil" in str(x) or "comod" in str(x))):
        return "PL8"
    elif ("avan" in str(x) or "progres" in str(x) or "aprobad" in str(x) or "pase" in str(x) or "evolucion" in str(x) or "cumplid" in str(x) or "realiz" in str(x) or "realis" in str(x) or "nota" in str(x) or "califica" in str(x)) and ("muestra" in str(x) or "regist" in str(x) or "exhibe" in str(x) or "present" in str(x) or "revela" in str(x) or "indica" in str(x) or "ver" in str(x) or "veo" in str(x) or "observa" in str(x)):
        return "PL9"
    elif ("funciona" in str(x) or "sirve" in str(x) or "desemp" in str(x) or "march" in str(x) or "traba" in str(x) or "trava" in str(x) or "estorba" in str(x) or "error" in str(x) or "falla" in str(x) or "fallo" in str(x) or "congela" in str(x) or "pasma" in str(x)) and ("material" in str(x) or "interac" in str(x) or "video" in str(x) or "tarjeta" in str(x) or "animado" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "animacion" in str(x) or "imagen" in str(x) or "grabacion" in str(x)):
        return "PL10"
    elif ("link" in str(x) or "enlace" in str(x) or "boton" in str(x) or "oprim" in str(x) or "hipervinculo" in str(x)) and ("descar" in str(x) or "bajar" in str(x) or "download" in str(x) or "materia" in str(x) or "foto" in str(x) or "fotografia" in str(x) or "video" in str(x) or "receta" in str(x)) and not ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL11"
    elif ("clase" in str(x) or "sesion" in str(x)) and ("traba" in str(x) or "carga" in str(x) or "borra" in str(x) or "funciona" in str(x) or "escucha" in str(x) or "oye" in str(x) or "lenta" in str(x) or "lento" in str(x) or "congel" in str(x)):
        return "PL12"
    elif ("chat" in str(x)) or (("mensaje" in str(x)) and ("app" in str(x) or "aplicacion" in str(x))):
        return "PL13"
    elif ("notifica" in str(x) or "alerta" in str(x) or "señal" in str(x) or "aviso" in str(x)) and ("plataforma" in str(x) or "moodle" in str(x) or "app" in str(x) or "aplicacion" in str(x) or "pagina" in str(x) or "internet" in str(x)):
        return "PL14"
    elif ("reprod" in str(x) or "abri" in str(x) or "desple" in str(x) or "abre" in str(x) or "inic" in str(x) or "escuch" in str(x) or "oir" in str(x) or "ver" in str(x)) and ("video" in str(x) or "audio" in str(x) or "multimedia" in str(x) or "sonido" in str(x)) and ("dispositivo" in str(x) or "celular" in str(x) or "telefo" in str(x) or "computa" in str(x) or "pc" in str(x) or "equipo" in str(x) or "iphone" in str(x) or "android" in str(x) or "mac" in str(x) or "tablet" in str(x) or "pad" in str(x)):
        return "PL15"
    elif ("congela" in str(x) or " traba " in str(x) or "se pone lenta" in str(x) or "esta lenta" in str(x) or "esta muy lenta" in str(x) or "pantalla" in str(x) or "trava" in str(x) or "esta lento" in str(x) or "va lenta" in str(x)):
        return "PL16"
    elif ("enlace" in str(x) or "link" in str(x) or "hipervinculo" in str(x) or "boton" in str(x) or "oprim" in str(x)) and ("roto" in str(x) or "corresponde" in str(x) or "parte" in str(x) or "sitio" in str(x)):
        return "PL17"  
    elif ("horario" in str(x) or " hora" in str(x) or "calendario" in str(x) or "flexible" in str(x) or "tarde" in str(x) or "noche" in str(x) or "madrugada" in str(x) or "dia" in str(x)) and ("atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "chef" in str(x) or "instructor" in str(x) or "maestro" in str(x) or "monitor" in str(x)):
        return "SE1"
    elif ("seguimiento" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxil" in str(x) or "apoyo" in str(x) or "colabor" in str(x) or "cooper" in str(x) or "atencion" in str(x) or "dispo" in str(x) or "guia" in str(x) or "atencion" in str(x) or "ayuda" in str(x) or "asistencia" in str(x) or "auxilio" in str(x) or "apoyo" in str(x) or "comunica" in str(x) or "habla" in str(x)) and ("linea" in str(x) or "numero" in str(x) or "what" in str(x) or "facebook" in str(x) or "instagram" in str(x)) and not ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)):
        return "SE2"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "instructor" in str(x) or "monitor" in str(x)) and ("otra area" in str(x) or "otro diplomado" in str(x) or "no sabe" in str(x) or "no tiene conocimiento" in str(x)):
        return "SE3"
    elif ("indu" in str(x)) and ("personal" in str(x) or "individual" in str(x) or "propio" in str(x) or "particular" in str(x) or "basico" in str(x) or "intermedio" in str(x) or "avanzado" in str(x) or "pc" in str(x) or "internet" in str(x) or "utilizar" in str(x) or "web" in str(x)):
        return "SE4"
    elif ("indu" in str(x)) and ("recibi" in str(x) or "dieron" in str(x) or "daron" in str(x) or "no pude" in str(x)):
        return "SE5"
    elif ("indu" in str(x)) and ("explica" in str(x) or "ensen" in str(x) or "aclara" in str(x) or "manifest" in str(x)):
        return "SE6"
    elif ("pacien" in str(x) or "seren" in str(x) or "tranquil" in str(x) or "lento" in str(x) or "despacio" in str(x)) and ("explica" in str(x) or "ensena" in str(x)):
        return "SE7"
    elif ("docente" in str(x) or "profe" in str(x) or "maestr" in str(x) or "chef" in str(x) or "tutor" in str(x)) and ("contacto" in str(x) or "numero" in str(x) or "telefono" in str(x) or "info de contacto" in str(x) or "informacion de contacto" in str(x) or "informacion para contactarse" in str(x)):
        return "SE8"
    elif ("llamada" in str(x) or "mensaje" in str(x) or "llama" in str(x)) and ("mucha" in str(x) or "demasia" in str(x) or "cantidad" in str(x) or "exceso" in str(x) or "monton" in str(x) or "manana" in str(x) or "tarde" in str(x) or "noche" in str(x) or "dia" in str(x)):
        return "SE9"
    elif ("termine" in str(x) or "acab" in str(x) or "complet" in str(x) or "conclu" in str(x) or "finali" in str(x)) and ("modulo" in str(x) or "curso" in str(x) or "diplomado" in str(x) or "escuela" in str(x) or "especiali" in str(x) or "carrera" in str(x)):
        return "SE10"
    elif ("seguimiento" in str(x) or "atencion" in str(x) or "atend" in str(x) or "preocup" in str(x)) and ("mas" in str(x) or "mayor" in str(x) or "aument" in str(x)):
        return "SE11"  
    elif ("duda" in str(x) or "pregunta" in str(x) or "cuestion" in str(x)) and ("esfuer" in str(x) or "interes" in str(x) or "ganas" in str(x)) and not ("interesante" in str(x)):
        return "SE12"
    elif ("profundizar" in str(x)) and ("temas" in str(x) or "asunto" in str(x) or "cosa" in str(x) or "agenda" in str(x) or "inscribir" in str(x) or "usar" in str(x) or "uso" in str(x) or "agendar clase" in str(x) or "plataforma" in str(x) or "retroalimentacion" in str(x)):
        return "SE13"
    elif ("tiempo" in str(x) or "tarda" in str(x) or "demora" in str(x)) and ("llamada" in str(x) or "llama" in str(x) or "chat" in str(x) or "spar" in str(x) or "contesta" in str(x) or "resp" in str(x) or "ayuda" in str(x) or "apoyo" in str(x) or "acompa" in str(x)):
        return "SE14"     
    elif ("nunca" in str(x) or "jamas" in str(x)) and ("duda" in str(x) or "pregunta" in str(x) or "respuesta" in str(x) or "colaboracion" in str(x) or "retroalimentacion" in str(x) or "contest" in str(x) or "llama" in str(x)):
        return "SE15"
    elif ("tiempo" in str(x) or "duracion" in str(x) or "dura" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)) and ("termina" in str(x) or "acaba" in str(x) or "culmin" in str(x) or "completa" in str(x) or "finali" or str(x) or "conclu" in str(x)):
        return "TD1"
    elif ("tiempo" in str(x) or "duracion" in str(x) or " hora" in str(x) or " dia" in str(x) or "periodo" in str(x) or "espacio" in str(x)) and ("entrega" in str(x) or "subi" in str(x) or "envia" in str(x) or " manda" in str(x)):
        return "TD2"
    elif ("pausa" in str(x) or "congela" in str(x) or "parar" in str(x) or "vaciones" in str(x) or "enfermedad" in str(x)) and ("diplomado" in str(x) or "carrera" in str(x) or "especializacion" in str(x) or "curso" in str(x) or "modulo" in str(x)):
        return "TD3"
    elif ("menos" in str(x) or "no mas" in str(x) or "muchas" in str(x) or "disminu" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR1"
    elif ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR2"
    elif ("sencill" in str(x) or "facil" in str(x) or "simple" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR3"
    elif ("tecnic" in str(x) or "teoric" in str(x) or "aplica" in str(x) or "videa real" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR4"
    elif ("dificil" in str(x) or "complica" in str(x) or "complej" in str(x) or "imposib" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR5"
    elif ("extens" in str(x) or "largo" in str(x) or "grande" in str(x) or "corto" in str(x) or "amplio" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR6"
    elif ("foto" in str(x) or "fotografia" in str(x) or "imagen" in str(x) or "prueba" in str(x)) and ("much" in str(x) or "demasi" in str(x) or "excesiv" in str(x) or "poco" in str(x)) and  ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR7"
    elif ("pregunta" in str(x) or "piden" in str(x) or "pedir" in str(x) or "exig" in str(x) or "solici" in str(x)) and ("se vieron" in str(x) or "se abordaron" in str(x) or "no explic" in str(x) or "no ensen" in str(x)):
        return "PR8"
    elif ("instrucc" in str(x) or "paso" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("clar" in str(x) or "confus" in str(x)):
        return "PR9"
    elif ("indice" in str(x) or "clasificacion" in str(x) or "seguimiento" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR11"
    elif ("tiempo" in str(x) or "espacio" in str(x) or "periodo" in str(x) or "dias" in str(x) or " hora" in str(x)) and ("real" in str(x) or "coincid" in str(x) or "igual" in str(x) or "en verdad" in str(x)):
        return "PR12"
    elif ("formato" in str(x) or "amigable" in str(x) or "forma" in str(x) or "disen" in str(x)) and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)):
        return "PR13"
    elif ("poder" in str(x) or "deberia" in str(x) or "seria" in str(x) or "opcion" in str(x) or "alternativ" in str(x) or "eleg" in str(x))  and ("actividad" in str(x) or "practica" in str(x) or "integradora" in str(x) or "ejercicio" in str(x) or "examen" in str(x) or "evaluacion" in str(x) or "tarea" in str(x) or "cuestionario" in str(x)) and ("video" in str(x) or "audio" in str(x) or "grabac" in str(x)):
        return "PR14"
    elif ("examen" in str(x) or "ejercicio" in str(x) or "test" in str(x) or "evalua" in str(x)) or ("confus" in str(x) or "confun" in str(x)):
        return "EC1"
    elif ("respuesta bien" in str(x) or "respuesta esta mal" in str(x) or "respuesta no coincide" in str(x) or "respuesta mal" in str(x) or "respuesta bien" in str(x) or "marque la respuesta" in str(x) or "marco la respuesta" in str(x) or "marco la opcion" in str(x)):
        return "EC1"
    elif ("instruccio" in str(x) or "ordene" in str(x) or "regla" in str(x)) and ("entend" in str(x) or "entiend" in str(x) or "comprend" in str(x)):
        return "EC2"
    elif ("pregun" in str(x)) and ("no explic" in str(x) or "no die" or "no hi" or "no est"):
        return "EC3"
    elif ("preguntas son" in str(x) or "preguntas debe" in str(x) or "preguntas mas" in str(x) or "preguntas menos" in str(x)) and ("dificil" in str(x) or "facil" in str(x) or "comple" in str(x) or "teor" in str(x) or "tecnic" in str(x)):
        return "EC4"
    elif ("mas eva" in str(x) or "mas eje" in str(x) or "mas cues" in str(x)):
        return "EC5"
    elif ("retroal" in str(x)) and ("evalua" in str(x) or "cuestion" in str(x)):
        return "EC6"
    elif ("receta" in str(x) or "formula" in str(x) or "lista" in str(x)) and ("mas" in str(x) or "aumenta" in str(x) or "incrementa" in str(x) or "amplia" in str(x)):
        return "RE1" #mas miremos a ver si se quita
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("ingrediente" in str(x) or "alimento" in str(x) or "comida" in str(x) or "componente" in str(x)):
        return "RE2" #mirar porque está repetida ahí arribe(
    elif ("sustitu" in str(x) or "reempla" in str(x) or "supli" in str(x) or "opciones" in str(x)) and ("utensilio" in str(x) or "util" in str(x) or "uten" in str(x) or "herramienta" in str(x) or "instrumento" in str(x) or "artefacto" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "oya" in str(x) or "sarten" in str(x) or "licuador" in str(x) or "cocina" in str(x) or "elemento" in str(x) or "tabla" in str(x) or "rallador" in str(x) or "pelador" in str(x) or "molde" in str(x) or "cacerola" in str(x)):
        return "RE3"
    elif (" tips" in str(x) or "truco" in str(x) or "atajo" in str(x) or "consejo" in str(x) or "recomenda" in str(x) or "sugerenci" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE5"
    elif ("actual" in str(x) or " nuev" in str(x) or " tenden" in str(x) or "presente" in str(x) or "vigente" in str(x) or "moda" in str(x) or "ultimas" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE6"
    elif ("porq" in str(x) or "por q" in str(x) or "motivo" in str(x) or "razon" in str(x) or "motivacion" in str(x) or "fundamento" in str(x) or "finalidad" in str(x)) and ("receta" in str(x) or "formula" in str(x)):
        return "RE7"
    elif ("receta" in str(x) or "formula" in str(x)) and ("especif" in str(x) or "distint" in str(x) or "especial" in str(x) or "particu" in str(x) or "diferen" in str(x) or "paso a paso" in str(x)):
        return "RE11"
    elif ("gramo" in str(x) or " tasa " in str(x) or " taza " in str(x) or "onza" in str(x) or " onsa" in str(x) or "kilo" in str(x) or "libra" in str(x)):
        return "RE9"
    elif ("receta" in str(x) or "formula" in str(x)) and ("descar" in str(x) or " baja" in str(x)):
        return "RE10"
    elif ("receta" in str(x) or "formula" in str(x)) and ("igual" in str(x) or "coinci" in str(x) or "concord" in str(x) or "similar" in str(x) or "equival" in str(x) or "mismo" in str(x)) and ("video" in str(x) or "grabac" in str(x) or "clase" in str(x)):
        return "RE12"
    elif ("paso a paso" in str(x) or "congruen" in str(x) or "cohere" in str(x)) and ("recet" in str(x) or "formula" in str(x)):
        return "RE13"
    elif ("tamano" in str(x) or "medida" in str(x) or "dimensio" in str(x) or "talla" in str(x) or "grande" in str(x) or "peque" in str(x)) and ("molde" in str(x) or "olla" in str(x) or "cacerola" in str(x) or "cuchillo" in str(x) or "tabla" in str(x) or "rallado" in str(x)):
        return "RE14"
    elif ("receta" in str(x)) and ("rubric" in str(x)):
        return "RE15"
    elif ("receta" in str(x) or "formula" in str(x)) and ("teoria" in str(x)):
        return "RE16"
    elif ("receta" in str(x) or "formula" in str(x) or "plato" in str(x) or "comida" in str(x) or "postre" in str(x)) and ("tiempo" in str(x) or "minuto" in str(x) or "hora" in str(x) or "duracion" in str(x) or "segundo" in str(x)):
        return "RE17"
    elif ("receta" in str(x) or "plato" in str(x) or "plato" in str(x) or "postre" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "pastel" in str(x) or "parrillada" in str(x) or "asado" in str(x)) and ("resultado" in str(x) or "final" in str(x) or "producto" in str(x) or "feo" in str(x) or "sabor" in str(x) or "quedo mal" in str(x) or "se ve mal" in str(x) or " sabe" in str(x) or "se ve feo" in str(x) or "no sale" in str(x) or "no sirve" in str(x)):
        return "RE19"
    elif ("receta" in str(x) or "formula" in str(x)) and ("diseno" in str(x) or "fotografia" in str(x) or " foto" in str(x) or " imagen" in str(x)):
        return "RE20"
    elif ("utensilio" in str(x) or "instrumento" in str(x) or "herramienta" in str(x) or "cuchillo" in str(x) or "olla" in str(x) or "molde" in str(x) or "rallador" in str(x) or "tijera" in str(x) or "olla" in str(x) or "batidor" in str(x) or "vaso" in str(x)) and ("explica" in str(x) or "ilustr" in str(x) or "ensen" in str(x) or "indica" in str(x) or "da clases" in str(x) or "educar" in str(x)):
        return "RE21"
    elif ("receta" in str(x) or "formula" in str(x) or "alimento" in str(x) or "comida" in str(x) or "bandeja" in str(x) or "parrill" in str(x) or "carne" in str(x) or "pollo" in str(x) or "pescado" in str(x) or "arroz" in str(x) or "postre" in str(x) or "torta" in str(x)) and ("difici" in str(x) or "complej" in str(x) or "complica" in str(x) or "imposibl" in str(x) or "arduo" in str(x) or "fácil" in str(x) or "sencill" in str(x) or "simple" in str(x)):
        return "RE22"
    else:
        return "-"

# Aplicación de la función a las sugerencias expuestas por los estudiantes

In [25]:
data_copia = data.copy()

In [26]:
#cambio los clasificadores
data_copia["Clasificacion"]=data_copia["Respuesta"].apply(lambda x: clasificador2_2(x))

In [27]:
data_copia[["Respuesta","Clasificacion"]].iloc[110:160]

,Respuesta,Clasificacion
110,ninguno,NO
111,no tengo sugerencias,NO
112,no tengo sugerencias,NO
113,nada,-
114,nada,-
115,nada,-
116,nada,-
117,nada,-
118,nada todo esta bien,NO
119,nada todo esta bien,NO


# Parte 2: Palabras clave

In [28]:
sugerencias = data_copia["Respuesta"]

# Definir funciones de unión y concatenación de sugerencias

In [29]:
def union(list):
    return " ".join(list)

In [30]:
def concatenar(list):
    result= " "
    for element in list:
        result += str(element)
    return result

# Se pasan las sugerencias a un texto unificado

Se pasan primero las sugerencias a lista y enseguida se une cada uno de los elementos de la lista para crear un texto unificado

In [31]:
list_sugerencias = sugerencias.values.tolist()

In [32]:
sugerencias_unidas = concatenar(list_sugerencias)

In [33]:
sugerencias_unidas

'                                           a mi me paresio de lo mejor  a mi si me a servido mucho chatear con mi docente muchas gracias al observar los videos se cortan ampliar plazo para terminar el curso actualizar los aprendizajes al principio lo mire como dificil  pero el chef rodrigo gallegos  me indico que no estaba usando el chocolate adecuado  gracias por todos los consejos que nos dan si algo me hubiese encantado que hicieramos  es un cake con glaseado tipo espejo  me matricule en este diplomado para aprender y poner en practica todo lo relacionado a pasteles y fue lo menos que encontre  pero he aprendido cosas que nunca imagine  gracias bien brindar un poco mas de imagenes para diferenciar los tonos de piel bueno solo faltaria un poco de mejor explicacion en los videos colocar la opcion de subir la foto mas clara colocar una alerta en el cuestionario para notificar una pregunta no contestada creo estuvo bien pero me falto mas videos especificamente creo que como lo han dise

# Se descargan las stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Se utilizan las stopwords del idioma español

In [35]:
stop_words = set(stopwords.words('spanish')) 
type(stop_words)

set

Se pasan las stopwords a lista

In [36]:
list_stop_words = list(stop_words)  

In [37]:
list_stop_words

['tuviera',
 'donde',
 'erais',
 'estuvo',
 'fuese',
 'antes',
 'tuviéramos',
 'estuvieses',
 'estando',
 'tuvieron',
 'tendrás',
 'tened',
 'que',
 'ya',
 'teníais',
 'habrías',
 'tuve',
 'lo',
 'están',
 'sentid',
 'haya',
 'vosotras',
 'vuestras',
 'les',
 'poco',
 'hubiste',
 'tengamos',
 'tuvierais',
 'habrán',
 'será',
 'estarás',
 'hubisteis',
 'estábamos',
 'habían',
 'en',
 'esa',
 'estoy',
 'fuéramos',
 'la',
 'esto',
 'hubiera',
 'estaban',
 'algo',
 'hubiese',
 'habéis',
 'serán',
 'era',
 'seamos',
 'éramos',
 'somos',
 'mucho',
 'fuera',
 'otra',
 'unos',
 'sus',
 'este',
 'sentidos',
 'estaréis',
 'para',
 'habréis',
 'no',
 'vuestro',
 'estar',
 'estéis',
 'fuésemos',
 'fueras',
 'estés',
 'estada',
 'hubieran',
 'pero',
 'fuiste',
 'tenemos',
 'habíais',
 'tendrías',
 'tuvieseis',
 'tendréis',
 'estabas',
 'del',
 'tuviésemos',
 'eso',
 'tuviesen',
 'algunas',
 'estuviéramos',
 'estuviera',
 'al',
 'seré',
 'habiendo',
 'estuvieras',
 'tuviese',
 'más',
 'otros',
 'ten

Se une los elementos de la lista list_stop_words en un solo texto

In [38]:
text_stop_words = union(list_stop_words)

In [39]:
text_stop_words

'tuviera donde erais estuvo fuese antes tuviéramos estuvieses estando tuvieron tendrás tened que ya teníais habrías tuve lo están sentid haya vosotras vuestras les poco hubiste tengamos tuvierais habrán será estarás hubisteis estábamos habían en esa estoy fuéramos la esto hubiera estaban algo hubiese habéis serán era seamos éramos somos mucho fuera otra unos sus este sentidos estaréis para habréis no vuestro estar estéis fuésemos fueras estés estada hubieran pero fuiste tenemos habíais tendrías tuvieseis tendréis estabas del tuviésemos eso tuviesen algunas estuviéramos estuviera al seré habiendo estuvieras tuviese más otros tengan habríais quienes durante hubiéramos sobre fuimos estuvierais muy hay habré estén estará nos ni nada otras también hubieses de he tendré una cual teníamos hayáis fuisteis el esta estamos habrás hasta ellas sería sentidas algunos eres tendremos fueses seríamos sin ha a eras desde había estuvisteis tendrían serías sintiendo estaremos habíamos estuviésemos su est

# Se remueven los acentos de las stopwords

In [40]:
text_stop_words_1 = unicodedata.normalize("NFKD", str(text_stop_words)).encode("ascii","ignore").decode("ascii")

In [41]:
text_stop_words_1 

'tuviera donde erais estuvo fuese antes tuvieramos estuvieses estando tuvieron tendras tened que ya teniais habrias tuve lo estan sentid haya vosotras vuestras les poco hubiste tengamos tuvierais habran sera estaras hubisteis estabamos habian en esa estoy fueramos la esto hubiera estaban algo hubiese habeis seran era seamos eramos somos mucho fuera otra unos sus este sentidos estareis para habreis no vuestro estar esteis fuesemos fueras estes estada hubieran pero fuiste tenemos habiais tendrias tuvieseis tendreis estabas del tuviesemos eso tuviesen algunas estuvieramos estuviera al sere habiendo estuvieras tuviese mas otros tengan habriais quienes durante hubieramos sobre fuimos estuvierais muy hay habre esten estara nos ni nada otras tambien hubieses de he tendre una cual teniamos hayais fuisteis el esta estamos habras hasta ellas seria sentidas algunos eres tendremos fueses seriamos sin ha a eras desde habia estuvisteis tendrian serias sintiendo estaremos habiamos estuviesemos su est

# Función de conversión a lista

In [42]:
def conversion_lista(string):
    li = list(string.split(" "))
    return li

# Texto Stopwords a lista

In [43]:
list_stop_words1 = conversion_lista(text_stop_words_1)

In [44]:
list_stop_words1 

['tuviera',
 'donde',
 'erais',
 'estuvo',
 'fuese',
 'antes',
 'tuvieramos',
 'estuvieses',
 'estando',
 'tuvieron',
 'tendras',
 'tened',
 'que',
 'ya',
 'teniais',
 'habrias',
 'tuve',
 'lo',
 'estan',
 'sentid',
 'haya',
 'vosotras',
 'vuestras',
 'les',
 'poco',
 'hubiste',
 'tengamos',
 'tuvierais',
 'habran',
 'sera',
 'estaras',
 'hubisteis',
 'estabamos',
 'habian',
 'en',
 'esa',
 'estoy',
 'fueramos',
 'la',
 'esto',
 'hubiera',
 'estaban',
 'algo',
 'hubiese',
 'habeis',
 'seran',
 'era',
 'seamos',
 'eramos',
 'somos',
 'mucho',
 'fuera',
 'otra',
 'unos',
 'sus',
 'este',
 'sentidos',
 'estareis',
 'para',
 'habreis',
 'no',
 'vuestro',
 'estar',
 'esteis',
 'fuesemos',
 'fueras',
 'estes',
 'estada',
 'hubieran',
 'pero',
 'fuiste',
 'tenemos',
 'habiais',
 'tendrias',
 'tuvieseis',
 'tendreis',
 'estabas',
 'del',
 'tuviesemos',
 'eso',
 'tuviesen',
 'algunas',
 'estuvieramos',
 'estuviera',
 'al',
 'sere',
 'habiendo',
 'estuvieras',
 'tuviese',
 'mas',
 'otros',
 'ten

# Agregamos palabras que queremos excluir del análisis 

In [45]:
list_stop_words1 += ["buenas","noches", "bien", "gracias", "excelente"]

# Función de eliminar StopWords

In [46]:
def eliminar_stop_words(texto, lista_stop_words):
    return ' '.join([word for word in texto.split(' ') if word not in lista_stop_words])

In [47]:
texto_limpio = eliminar_stop_words(sugerencias_unidas,list_stop_words1)

In [48]:
texto_limpio

'                                           paresio mejor  servido chatear docente muchas observar videos cortan ampliar plazo terminar curso actualizar aprendizajes principio mire dificil  chef rodrigo gallegos  indico usando chocolate adecuado  consejos dan encantado hicieramos  cake glaseado tipo espejo  matricule diplomado aprender poner practica relacionado pasteles menos encontre  aprendido cosas nunca imagine  brindar imagenes diferenciar tonos piel bueno solo faltaria mejor explicacion videos colocar opcion subir foto clara colocar alerta cuestionario notificar pregunta contestada creo falto videos especificamente creo disenado   curso bueno  leer ir comprendiendo  dar ejemplos visuales  manera gerneral preguntas examen tocadas caso engranajes reduccion velocidad motor dice caja velocidad aumenta traccion pregunta ambigua  sabemos relacion caja velocidad motor ver modulo motor estudiando  sugerencia general dar opcion descargar videos clases evaluar horario 2 3 horario laboral 

# Mirar frecuencia de diferentes palabras

In [49]:
palabras=texto_limpio.split(" ")

Contar el número de repeticiones por palabra

In [50]:
contador=Counter(palabras)

# Mirar las palabras más repetidas

In [51]:
top_palabras=contador.most_common(500)
top_palabras_frame=pd.DataFrame(top_palabras)
print(top_palabras)

[('', 5793), ('videos', 645), ('curso', 368), ('clases', 330), ('vivo', 264), ('practica', 247), ('solo', 217), ('informacion', 203), ('modulo', 198), ('video', 182), ('creo', 171), ('perfecto', 168), ('muchas', 162), ('bueno', 162), ('hacer', 161), ('tiempo', 161), ('practicas', 160), ('recetas', 159), ('parece', 157), ('cada', 155), ('material', 135), ('momento', 134), ('gustaria', 134), ('clase', 131), ('poder', 121), ('tener', 119), ('asi', 119), ('parecio', 114), ('mejor', 113), ('pdf', 109), ('explicado', 108), ('gusto', 104), ('interesante', 101), ('ninguna', 98), ('buena', 96), ('poner', 95), ('paso', 95), ('preguntas', 94), ('ver', 92), ('mejorar', 91), ('trabajo', 90), ('ser', 90), ('ahora', 88), ('q', 85), ('ejemplos', 83), ('contenido', 83), ('receta', 83), ('claro', 82), ('actividades', 82), ('pues', 79), ('actividad', 78), ('buen', 76), ('completo', 74), ('sugiero', 74), ('ayuda', 74), ('ninguno', 73), ('aprendido', 72), ('deberian', 72), ('sugerencias', 71), ('tema', 71)

# Parte 3: escoger aleatoriamente ciertas clasificaciones a partir del código de clasificación

# Mirar como están distribuidas las clasificaciones de acuerdo a ciertas palabras claves

# Definir función de probabilidades

In [52]:
def probabilidad(list):
    return [i/sum(list) for i in list]

# Ejemplo para descarga

In [53]:
#c_descarga=data_copia[data_copia["Respuesta"].
           #str.contains("descarga")]["Clasificacion"].value_counts().to_frame()
#reset_index me pasa el indice a columna
#c_descarga.reset_index(inplace=True)
#se cambia el nombre a las columnas
#c_descarga.columns = ["clasificacion","repeticiones"]
#filtar el dataframe
#f_descarga=c_descarga[c_descarga["clasificacion"]!="-"]

In [54]:
#la izquierda hace referencia a filas, la derecha, la columna de selección
#cla_descarga=f_descarga.iloc[:4,0].tolist()
#fre_descarga=f_descarga.iloc[:4,1].tolist()

Lista de probabilidades con la función probabilidad aplicada

In [55]:
#pro_descarga=probabilidad(fre_descarga)
#pro_descarga

# Ejemplo varias palabras

In [56]:
#videos
#c_videos=data_copia[data_copia["Respuesta"].
#          str.contains("videos")]["Clasificacion"].value_counts().to_frame()
#c_videos.reset_index(inplace=True)
#c_videos.columns = ["clasificacion","repeticiones"]
#f_videos=c_videos[c_videos["clasificacion"]!="-"]
#cla_videos=f_videos.iloc[:5,0].tolist()
#fre_videos=f_videos.iloc[:5,1].tolist()
#pro_videos=probabilidad(fre_videos)

In [57]:
#clases
#c_clases=data_copia[data_copia["Respuesta"].
#           str.contains("clase")]["Clasificacion"].value_counts().to_frame()
#c_clases.reset_index(inplace=True)
#c_clases.columns = ["clasificacion","repeticiones"]
#f_clases=c_clases[c_clases["clasificacion"]!="-"]
#cla_clases=f_clases.iloc[:5,0].tolist()
#fre_clases=f_clases.iloc[:5,1].tolist()
#pro_clases=probabilidad(fre_clases)

In [58]:
#practica
#c_practica=data_copia[data_copia["Respuesta"].
#           str.contains("practica")]["Clasificacion"].value_counts().to_frame()
#c_practica.reset_index(inplace=True)
#c_practica.columns = ["clasificacion","repeticiones"]
#f_practica=c_practica[c_practica["clasificacion"]!="-"]
#f_practica=c_practica[(c_practica.clasificacion != "NO") & (c_practica.clasificacion !="-")]
#cla_practica=f_practica.iloc[:5,0].tolist()
#fre_practica=f_practica.iloc[:5,1].tolist()
#pro_practica=probabilidad(fre_practica)

# Función que crea variables para n palabras

In [59]:
lista_palabras=["descarga","video", "clases","practica","modulo","informacion","tiempo","video","practicas","receta",
               "material","pdf","actividad","ver","aprender","tema","ayuda",
               "trabajo","contenido","ejemplo","diplomado","duda","aprendizaje",
               "explicacion","apoyo","respuesta","docente","foto","modulo",
               "pregunta","ingrediente","dificil","docente","retroalimentacion",
               "chef","tips","teoria","tecnica","pregunta","util","leccion","chef",
               "leer","evaluacion","imagen","audio","hora","escrito","pasos",
               "examen","contesta","practico","sistema","conocimiento","proceso",
               "integradora","interactivo","virtual","horario","profesor","cuestionario",
               "profesor","cuestionario","internet","archivo","whatsapp","costo"]

In [60]:
for palabra in lista_palabras:
    locals()["c_" + str(palabra)]=c_videos=data_copia[data_copia["Respuesta"].
           str.contains(palabra)]["Clasificacion"].value_counts().to_frame()
    locals()["c_" + str(palabra)].reset_index(inplace=True)
    locals()["c_" + str(palabra)].columns = ["clasificacion","repeticiones"]
    #locals()["f_" + str(palabra)]=locals()["c_" + str(palabra)][locals()["c_" + str(palabra)]["clasificacion"]!="-"]
    locals()["f_" + str(palabra)]=locals()["c_" + str(palabra)][(locals()["c_" + str(palabra)].clasificacion != "NO")&(locals()["c_" + str(palabra)].clasificacion != "-")]
    locals()["cla_" + str(palabra)]=locals()["f_" + str(palabra)].iloc[:5,0].tolist()
    locals()["fre_" + str(palabra)]=locals()["f_" + str(palabra)].iloc[:5,1].tolist()
    locals()["pro_" + str(palabra)]=probabilidad(locals()["fre_" + str(palabra)])

In [61]:
#Verificacion vector se sugerencias
cla_video

['VI4', 'VI9', 'VI1', 'VI6', 'VI15']

In [62]:
#Verificación vector de probabilidades para cada sugerencia segun palabra
pro_video

[0.5188470066518847,
 0.2017738359201774,
 0.15964523281596452,
 0.06208425720620843,
 0.057649667405764965]

In [63]:
#mirar sugerencia y en que clasificacion esta si CONTIENE la palabra ____
data_copia[data_copia["Respuesta"].
           str.contains("se corta")][["Respuesta","Clasificacion"]].iloc[0:50]

,Respuesta,Clasificacion
25,al observar los videos se cortan,-
147,se cortan los videos,-
515,hay ocasiones que los videos se cortar no se ...,-
1554,deberian chequear los video por no estan comp...,CO11
3156,algunos videos se cortan al final,-
3545,uno de los videos no esta completo se corta ...,-
3954,sugerencia que los videos del curso 2 se pue...,VI15
5301,me hubiera gustado contar con mas opciones de...,PL12


In [64]:
data_copia[data_copia["Respuesta"].
           str.contains("video")]["Clasificacion"].value_counts().to_frame()

,Clasificacion
VI4,234
-,100
VI9,91
VI1,72
VI6,28
VI15,26
VI7,26
VI11,22
VI14,17
PL6,15


# Se copia la base datos para ver que tal se comporta con el clasificador3(x,y)

In [65]:
data_copia1 = data_copia.copy()

# Código que escoge sugerencias a partir de la probabilidad

In [66]:
def clasificador3(x,y):
    if "descarga" in str(x) and "-" in str(y):
        return np.random.choice(cla_descarga,p=pro_descarga)
    elif "video" in str(x) and "-" in str(y):
        return np.random.choice(cla_video,p=pro_video)
    elif "clases" in str(x) and "-" in str(y):
        return np.random.choice(cla_clases,p=pro_clases)
    elif "practica" in str(x) and "-" in str(y):
        return np.random.choice(cla_practica,p=pro_practica)
    elif "modulo" in str(x) and "-" in str(y):
        return np.random.choice(cla_modulo,p=pro_modulo)
    elif "informacion" in str(x) and "-" in str(y):
        return np.random.choice(cla_informacion,p=pro_informacion)
    elif "tiempo" in str(x) and "-" in str(y):
        return np.random.choice(cla_tiempo,p=pro_tiempo)
    elif "video" in str(x) and "-" in str(y):
        return np.random.choice(cla_video,p=pro_video)
    elif "practicas" in str(x) and "-" in str(y):
        return np.random.choice(cla_practicas,p=pro_practicas)
    elif "receta" in str(x) and "-" in str(y):
        return np.random.choice(cla_receta,p=pro_receta)
    elif "material" in str(x) and "-" in str(y):
        return np.random.choice(cla_material,p=pro_material)
    elif "pdf" in str(x) and "-" in str(y):
        return np.random.choice(cla_pdf,p=pro_pdf)
    elif "actividad" in str(x) and "-" in str(y):
        return np.random.choice(cla_actividad,p=pro_actividad)
    elif "ver" in str(x) and "-" in str(y):
        return np.random.choice(cla_ver,p=pro_ver)
    elif "aprender" in str(x) and "-" in str(y):
        return np.random.choice(cla_aprender,p=pro_aprender)
    elif "tema" in str(x) and "-" in str(y):
        return np.random.choice(cla_tema,p=pro_tema)
    elif "ayuda" in str(x) and "-" in str(y):
        return np.random.choice(cla_ayuda,p=pro_ayuda)
    elif "trabajo" in str(x) and "-" in str(y):
        return np.random.choice(cla_trabajo,p=pro_trabajo)
    elif "contenido" in str(x) and "-" in str(y):
        return np.random.choice(cla_contenido,p=pro_contenido)
    elif "ejemplo" in str(x) and "-" in str(y):
        return np.random.choice(cla_ejemplo,p=pro_ejemplo)
    elif "diplomado" in str(x) and "-" in str(y):
        return np.random.choice(cla_diplomado,p=pro_diplomado)
    elif "duda" in str(x) and "-" in str(y):
        return np.random.choice(cla_duda,p=pro_duda)
    elif "aprendizaje" in str(x) and "-" in str(y):
        return np.random.choice(cla_aprendizaje,p=pro_aprendizaje)
    elif "explicacion" in str(x) and "-" in str(y):
        return np.random.choice(cla_explicacion,p=pro_explicacion)
    elif "apoyo" in str(x) and "-" in str(y):
        return np.random.choice(["CO3","DO2"],p=[0.6,0.4])
    elif "respuesta" in str(x) and "-" in str(y):
        return np.random.choice(cla_respuesta,p=pro_respuesta)
    elif "docente" in str(x) and "-" in str(y):
        return np.random.choice(cla_docente,p=pro_docente)
    elif "foto" in str(x) and "-" in str(y):
        return np.random.choice(cla_foto,p=pro_foto)
    elif "modulo" in str(x) and "-" in str(y):
        return np.random.choice(cla_modulo,p=pro_modulo)
    elif "ingrediente" in str(x) and "-" in str(y):
        return np.random.choice(cla_ingrediente,p=pro_ingrediente)
    elif "dificil" in str(x) and "-" in str(y):
        return np.random.choice(cla_dificil,p=pro_dificil)
    elif "docente" in str(x) and "-" in str(y):
        return np.random.choice(cla_docente,p=pro_docente)
    elif "retroalimentacion" in str(x) and "-" in str(y):
        return np.random.choice(cla_retroalimentacion,p=pro_retroalimentacion)
    elif "chef" in str(x) and "-" in str(y):
        return np.random.choice(cla_chef,p=pro_chef)
    elif "tips" in str(x) and "-" in str(y):
        return np.random.choice(cla_tips,p=pro_tips)
    elif "teoria" in str(x) and "-" in str(y):
        return np.random.choice(cla_teoria,p=pro_teoria)
    elif "tecnica" in str(x) and "-" in str(y):
        return np.random.choice(cla_tecnica,p=pro_tecnica)
    elif "pregunta" in str(x) and "-" in str(y):
        return np.random.choice(cla_pregunta,p=pro_pregunta)
    elif "util" in str(x) and "-" in str(y):
        return np.random.choice(cla_util,p=pro_util)
    elif "leccion" in str(x) and "-" in str(y):
        return np.random.choice(cla_leccion,p=pro_leccion)
    elif "leer" in str(x) and "-" in str(y):
        return np.random.choice(cla_leer,p=pro_leer)
    elif "evaluacion" in str(x) and "-" in str(y):
        return np.random.choice(cla_evaluacion,p=pro_evaluacion)
    elif "imagen" in str(x) and "-" in str(y):
        return np.random.choice(cla_imagen,p=pro_imagen)
    elif "audio" in str(x) and "-" in str(y):
        return np.random.choice(cla_audio,p=pro_audio)
    elif " hora" in str(x) and "-" in str(y):
        return np.random.choice(cla_hora,p=pro_hora)
    elif "escrito" in str(x) and "-" in str(y):
        return np.random.choice(cla_escrito,p=pro_escrito)
    elif "pasos" in str(x) and "-" in str(y):
        return np.random.choice(cla_pasos,p=pro_pasos)
    elif "examen" in str(x) and "-" in str(y):
        return np.random.choice(cla_contesta,p=pro_practico)
    elif "contesta" in str(x) and "-" in str(y):
        return np.random.choice(cla_contesta,p=pro_contesta)
    elif "practico" in str(x) and "-" in str(y):
        return np.random.choice(cla_practico,p=pro_practico)
    elif "sistema" in str(x) and "-" in str(y):
        return np.random.choice(cla_sistema,p=pro_sistema)
    elif "conocimiento" in str(x) and "-" in str(y):
        return np.random.choice(cla_conocimiento,p=pro_conocimiento)
    elif "proceso" in str(x) and "-" in str(y):
        return np.random.choice(cla_proceso,p=pro_proceso)
    elif "integradora" in str(x) and "-" in str(y):
        return np.random.choice(cla_integradora,p=pro_integradora)
    elif "interactivo" in str(x) and "-" in str(y):
        return np.random.choice(cla_interactivo,p=pro_interactivo)
    elif "virtual" in str(x) and "-" in str(y):
        return np.random.choice(cla_virtual,p=pro_virtual)
    elif "horario" in str(x) and "-" in str(y):
        return np.random.choice(cla_horario,p=pro_profesor)
    elif "cuestionario" in str(x) and "-" in str(y):
        return np.random.choice(cla_cuestionario,p=pro_cuestionario)
    elif "profesor" in str(x) and "-" in str(y):
        return np.random.choice(cla_profesor,p=pro_profesor)
    elif "internet" in str(x) and "-" in str(y):
        return np.random.choice(cla_internet,p=pro_internet)
    elif "archivo" in str(x) and "-" in str(y):
        return np.random.choice(cla_archivo,p=pro_archivo)
    elif "whatsapp" in str(x) and "-" in str(y):
        return np.random.choice(cla_whatsapp,p=pro_whatsapp)
    elif "costo" in str(x) and "-" in str(y):
        return np.random.choice(cla_costo,p=pro_costo)
    else:
        return str(y)

In [67]:
x=data_copia1
data_copia1["Micro"]=data_copia1.apply(lambda x: clasificador3(x["Respuesta"],x["Clasificacion"]), axis=1)

In [68]:
data_copia1[data_copia1["Respuesta"].
           str.contains("video")]["Micro"].value_counts().to_frame()

,Micro
VI4,281
VI9,113
VI1,91
VI15,33
VI6,33
VI7,26
VI11,22
VI14,17
VI5,15
PL6,15


In [69]:
data_copia1[data_copia1["Respuesta"].
           str.contains("se corta")][["Respuesta","Micro","Diplomado"]].iloc[0:50]

,Respuesta,Micro,Diplomado
25,al observar los videos se cortan,VI4,Repostería Profesional
147,se cortan los videos,VI15,Repostería Profesional
515,hay ocasiones que los videos se cortar no se ...,VI4,Repostería y Pastelería
1554,deberian chequear los video por no estan comp...,CO11,Diplomado en Nutrición y Buena Alimentación
3156,algunos videos se cortan al final,VI9,Diplomado en Nutrición y Buena Alimentación
3545,uno de los videos no esta completo se corta ...,VI15,Diplomado en Nutrición y Buena Alimentación
3954,sugerencia que los videos del curso 2 se pue...,VI15,NaN
5301,me hubiera gustado contar con mas opciones de...,PL12,Repostería y Pastelería


In [70]:
data_copia1[data_copia1["Diplomado"]=="Repostería Profesional"].iloc[0:50]

,Fecha,Respuesta,Diplomado,Version,Modulo,Id,Email,Recuento,Unnamed: 8,Unnamed: 9,Clasificacion,Micro
5,2021-04-01 00:00:00,,Repostería Profesional,V2,Nueces y semillas,0035G00001f7urLQAQ,guerrerorodriguezb5@gmail.com,1,NaN,NaN,-,-
23,2021-04-01 00:00:00,a mi me paresio de lo mejor,Repostería Profesional,V2,Muffins de frutos rojos,0035G00001gtdj4QAA,maritzapalma_20@yahoo.com,1,NaN,NaN,-,-
24,2021-04-01 00:00:00,a mi si me a servido mucho chatear con mi doc...,Repostería Profesional,V2,Muffins de frutos rojos,0035G00001f9iuhQAA,yashirayfrey@gmail.com,1,NaN,NaN,NO,NO
25,2021-04-01 00:00:00,al observar los videos se cortan,Repostería Profesional,V2,Jarabes y caramelos,0035G00001f93FVQAY,jamicaceres2707@gmail.com,1,NaN,NaN,-,VI4
43,2021-04-01 00:00:00,en las clases en vivo que el profesor o moder...,Repostería Profesional,V2,Higiene y herramientas,0035G00001gs1v3QAA,susy83@hotmail.com,1,NaN,NaN,DO3,DO3
49,2021-04-01 00:00:00,especificar que hacer cuando ocurren ciertas ...,Repostería Profesional,V2,Crème Brûlée,003f400001VWlRfAAL,daniv_99@hotmail.com,1,NaN,NaN,-,VI4
55,2021-04-01 00:00:00,excelente,Repostería Profesional,V2,Muffins de frutos rojos,0035G00001grufKQAQ,jesacuaya@gmail.com,1,NaN,NaN,NO,NO
73,2021-04-01 00:00:00,los chefs en las clases en vivo deberian refo...,Repostería Profesional,V2,Frutas y hierbas,0035G00001grufKQAQ,jesacuaya@gmail.com,1,NaN,NaN,CV4,CV4
76,2021-04-01 00:00:00,me parecio una practica muy interesante y sob...,Repostería Profesional,V2,Panqué de nuez,0035G00001dzwDZQAY,diancamro@gmail.com,1,NaN,NaN,NO,NO
80,2021-04-01 00:00:00,mas videos de recetas,Repostería Profesional,V2,Higiene y herramientas,0035G00001gsEW3QAM,mariigm13@gmail.com,1,NaN,NaN,VI1,VI1


In [71]:
data_copia1.columns

Index(['Fecha', 'Respuesta', 'Diplomado', 'Version', 'Modulo', 'Id', 'Email',
       'Recuento', 'Unnamed: 8', 'Unnamed: 9', 'Clasificacion', 'Micro'],
      dtype='object')

# Función de clasificación macrocategorías

In [72]:
def macro(x):
    if "CO" in str(x):
        return "Contenido"
    elif "CV" in str(x):
        return "Clases en vivo"
    elif "COV" in str(x):
        return "Covid"
    elif "DO" in str(x):
        return "Docencia"
    elif "OT" in str(x):
        return "Otros"
    elif "PL" in str(x):
        return "Plataforma"
    elif "SE" in str(x):
        return "Servicio"
    elif "TD" in str(x):
        return "Tiempo del diplomado"
    elif "PR" in str(x):
        return "AP/EP"
    elif "EC" in str(x):
        return "Ejercicios/cuestionario"
    elif "RE" in str(x):
        return "Recetas"
    elif "VI" in str(x):
        return "Videos"
    elif "AU" in str(x):
        return "Audios"
    else:
        return "N/A"

# Aplicación función macrocategorías

In [73]:
data_copia1["Macro"]=data_copia1["Micro"].apply(lambda x: macro(x))

In [74]:
data_copia1[["Micro","Macro"]].head(n=50)

,Micro,Macro
0,-,N/A
1,-,N/A
2,-,N/A
3,-,N/A
4,-,N/A
5,-,N/A
6,-,N/A
7,-,N/A
8,-,N/A
9,-,N/A


In [75]:
data_copia1.columns

Index(['Fecha', 'Respuesta', 'Diplomado', 'Version', 'Modulo', 'Id', 'Email',
       'Recuento', 'Unnamed: 8', 'Unnamed: 9', 'Clasificacion', 'Micro',
       'Macro'],
      dtype='object')

In [76]:
### ELEGIR ###
#data_final = data_copia1[["Respuesta","Diplomado","Modulo","Micro","Macro"]]
data_final = data_copia1[["Respuesta","Diplomado","Version","Modulo","Micro","Macro"]]

In [77]:
data_final.iloc[150:200]

,Respuesta,Diplomado,Version,Modulo,Micro,Macro
150,seria bueno que tambien nos den ejemplo como ...,Organización de Eventos,V2,"Grupos sociales, nuevas tendencias y medios",PL6,Plataforma
151,seria de gran valor un ejemplo de las hojas y...,Apertura de Negocio de Alimentos y Bebidas,V2,“P&G” o “Estado de Pérdidas y Ganancias”,NO,N/A
152,siento que estoy desconectada tengo muchas a...,Técnicas Culinarias,V2,Seguridad y Herramientas,PR1,AP/EP
153,sin comentarios,Inteligencia Emocional y Psicología Positiva,V3,Mindfulness (atención plena),NO,N/A
154,sin comentarios,Nutrición y Salud,V2,Cargando kilos de más,NO,N/A
155,sin comentarios me parecio excelente,Instalaciones de Energía Solar,V4,Energía solar,NO,N/A
156,sugerir rutinas de entrenamiento concurrente ...,Entrenador Personal,V3,Respuestas y adaptaciones al ejercicio,EC1,Ejercicios/cuestionario
157,sugiero dividir la practica integradora en pa...,Nutrición y Salud,V2,Asistencia nutricional,DO8,Docencia
158,tener una linea solo para docentes,Nutrición y Salud,V2,Triada del atleta,DO3,Docencia
159,todo genial,Nutrición y Buena Alimentación,V2,Práctica Integradora Final,NO,N/A


In [78]:
data_final[data_final["Diplomado"]=="Repostería y Pastelería"]

,Respuesta,Diplomado,Version,Modulo,Micro,Macro
4,,Repostería y Pastelería,V2,Pastelería 2,-,N/A
28,al principio lo mire como dificil pero el ch...,Repostería y Pastelería,V2,Chocolatería 2,CV3,Clases en vivo
37,dar mas ejemplos visuales,Repostería y Pastelería,V2,Panadería 1,EC5,Ejercicios/cuestionario
39,deberian permitir descargar los videos de tod...,Repostería y Pastelería,V2,Panadería 2,PL6,Plataforma
41,den la opcion de descargar los videos como pa...,Repostería y Pastelería,V2,Hojaldre y Pàte á Choux,PL6,Plataforma
...,...,...,...,...,...,...
5944,excelente,Repostería y Pastelería,V2,Mousses y bavaresas,NO,N/A
5952,mas videos,Repostería y Pastelería,V2,Pastelería 2,VI4,Videos
5955,me parece muy completa sin embargo se podria...,Repostería y Pastelería,V2,Napoleón,PL9,Plataforma
5995,todo perfecto,Repostería y Pastelería,V2,Mousses y bavaresas,NO,N/A


# Importacion de la data escuela y versión de los cursos

In [79]:
##### ELEGIR #####
#data_diplomados = pd.read_excel("C:/Users/felip/Desktop/Trabajo/Clasificación Pain Points/diplomados.xlsx")
data_diplomados = pd.read_excel("C:/Users/felip/Desktop/Trabajo/Clasificación Pain Points/diplomados_sinv.xlsx")

In [80]:
data_diplomados.head()

,Diplomado,Escuela
0,Administración de Restaurantes,NEGOCIOS Y EMPRENDIMIENTO
1,Alimentación Vegana y Vegetariana,SALUD Y BIENESTAR
2,Apertura de Negocio de Alimentos y Bebidas,NEGOCIOS Y EMPRENDIMIENTO
3,Catering,NEGOCIOS Y EMPRENDIMIENTO
4,Cocina Internacional,GASTRONOMÍA


# Cruce de la data análisis con la data de diplomado

In [81]:
micro_explicacion = pd.read_excel("C:/Users/felip/Desktop/Trabajo/Clasificación Pain Points/micro_explicacion.xlsx")

In [82]:
data_analisis01 = pd.merge(data_final, 
                      data_diplomados, 
                      on ='Diplomado', 
                      how ='inner')

In [83]:
data_analisis = pd.merge(data_analisis01,micro_explicacion, on = "Micro", how="inner")

In [84]:
data_analisis.iloc[400:450]

,Respuesta,Diplomado,Version,Modulo,Micro,Macro,Escuela,Micro categoria
400,que las clases fueran en horarios matutinos p...,Meditación Mindfulness,V3,¿Qué es mindfulness?,CV2,Clases en vivo,SALUD Y BIENESTAR,Horarios más flexibles
401,que las clases sean mas animadas e instructiv...,Plomería,V3,Introducción a la plomería y fontanería,CV2,Clases en vivo,OFICIOS,Horarios más flexibles
402,son mis primeros clases aun no se que opinar,Corte y Confección,V1,Introducción al corte y confección,CV2,Clases en vivo,BELLEZA Y MODA,Horarios más flexibles
403,bueno por lo menos yo personalmente la dificu...,Corte y Confección,V1,Introducción al corte y confección,CV2,Clases en vivo,BELLEZA Y MODA,Horarios más flexibles
404,todo me parece acorde a la clases,Producción de Eventos Especializados,V2,Los 15 años,CV2,Clases en vivo,NEGOCIOS Y EMPRENDIMIENTO,Horarios más flexibles
405,mas clases en vivo,Producción de Eventos Especializados,V2,Los 15 años,CV2,Clases en vivo,NEGOCIOS Y EMPRENDIMIENTO,Horarios más flexibles
406,incluir las clases en vivo,Cuidado del Adulto Mayor,V2,Cuidados paliativos,CV2,Clases en vivo,SALUD Y BIENESTAR,Horarios más flexibles
407,lo mas mejor serian clases en vivo,Cuidado del Adulto Mayor,V2,Estrategias claves para el buen cuidado del ad...,CV2,Clases en vivo,SALUD Y BIENESTAR,Horarios más flexibles
408,clases en vivo,Cuidado del Adulto Mayor,V2,Estrategias claves para el buen cuidado del ad...,CV2,Clases en vivo,SALUD Y BIENESTAR,Horarios más flexibles
409,con mas clases en vivo,Cuidado del Adulto Mayor,V2,Generalidades relacionadas al cuidado del adul...,CV2,Clases en vivo,SALUD Y BIENESTAR,Horarios más flexibles


In [85]:
sorted(list(data_analisis["Diplomado"].unique().tolist()))

['Administración de Restaurantes',
 'Alimentación Vegana y Vegetariana',
 'Apertura de Negocio de Alimentos y Bebidas',
 'Catering',
 'Cocina Internacional',
 'Cocina Tradicional Mexicana',
 'Corte y Confección',
 'Cuidado del Adulto Mayor',
 'Diplomado en Nutrición y Buena Alimentación',
 'Energía Eólica',
 'Energía Solar',
 'Instalaciones de Energía Solar',
 'Instalación y mantenimiento de aires acondicionados',
 'Inteligencia Emocional y Psicología Positiva',
 'Manicure',
 'Maquillaje',
 'Maquillaje Social',
 'Marketing para emprendedores',
 'Mecánica Automotriz',
 'Mecánica de Motos',
 'Meditación Mindfulness',
 'Nutrición y Buena Alimentación',
 'Nutrición y Salud',
 'Organización de Eventos',
 'Parrilladas y Asados',
 'Plomería',
 'Producción de Eventos Especializados',
 'Reparación Electrónica',
 'Reparación y Mantenimiento de Celulares',
 'Repostería Profesional',
 'Repostería y Pastelería',
 'Todo Sobre Vinos',
 'Técnicas Culinarias',
 'Vitivinicultura y Cata de Vinos']

In [86]:
data_analisis[data_analisis["Diplomado"]=="Repostería y Pastelería"]

,Respuesta,Diplomado,Version,Modulo,Micro,Macro,Escuela,Micro categoria
33,esta fue mi practica favorita gracias,Repostería y Pastelería,V2,Chocolatería 1,VI9,Videos,GASTRONOMÍA,Más AP/EP en video
34,todo estuvo claro y la practica magistral,Repostería y Pastelería,V2,Angel Food,VI9,Videos,GASTRONOMÍA,Más AP/EP en video
35,son buenas tecnicas de ensenanzas y creo esta...,Repostería y Pastelería,V2,Mousse de chocolate a base de crema inglesa.,VI9,Videos,GASTRONOMÍA,Más AP/EP en video
36,que solo se realice un video de la practica i...,Repostería y Pastelería,V2,Pastelería 2,VI9,Videos,GASTRONOMÍA,Más AP/EP en video
37,buenas practicas los pdf no son muy buenos p...,Repostería y Pastelería,V2,Napoleón,VI9,Videos,GASTRONOMÍA,Más AP/EP en video
...,...,...,...,...,...,...,...,...
2208,me dio la oportunidad de aplicar diversos con...,Repostería y Pastelería,V2,Angel Food,TD1,Tiempo del diplomado,GASTRONOMÍA,Más tiempo para terminar el diplomado
2214,bueno yo beo que todo esta muy bien explican ...,Repostería y Pastelería,V2,Napoleón,DO17,Docencia,GASTRONOMÍA,Hablar más lento y claro en llamadas telefónic...
2215,me ayudaria mucho mas si usan los numeros y n...,Repostería y Pastelería,V2,Panadería 1,SE2,Servicio,GASTRONOMÍA,Tener más líneas de atención para comunicarse
2217,poner mas opciones ne caso de no tenerse el i...,Repostería y Pastelería,V2,Mousse de chocolate a base de crema inglesa.,RE2,Recetas,GASTRONOMÍA,Tener opciones de sustitución de ingredientes


In [87]:
data_analisis_final = data_analisis[data_analisis["Macro"]!="N/A"]

In [88]:
len(data_analisis_final)

2304

In [89]:
data_prueba=data_analisis_final.iloc[0:500]
len(data_prueba)

500

# Escribir un documento XLSX

In [90]:
data_analisis_final.to_excel("data_analisis_final.xlsx",index=False)

# Documentos de palabras clave

In [91]:
top_palabras_frame.columns=["Palabras","Repeticiones"]

In [92]:
top_palabras_frame

,Palabras,Repeticiones
0,,5793
1,videos,645
2,curso,368
3,clases,330
4,vivo,264
...,...,...
495,usan,10
496,trabajando,10
497,contacto,10
498,historia,10


In [93]:
ranking_palabras=top_palabras_frame.iloc[1:]

In [94]:
ranking_palabras.to_excel("ranking_palabras.xlsx",sheet_name='Hoja1',index=False)